In [ ]:
# ============================================================================
# CELDA 1: CONFIGURACIÓN E IMPORTS
# ============================================================================

import os
import json
import re
import time
import requests
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import PyPDF2
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Markdown
import anthropic
from dotenv import load_dotenv

# Configuración
load_dotenv(Path('.env') if Path('.env').exists() else Path('../.env'))

# Verificar API keys
PDF_REST_API_KEY = os.getenv('PDF_REST_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

if not PDF_REST_API_KEY:
    raise ValueError("❌ PDF_REST_API_KEY no encontrada en .env")
if not ANTHROPIC_API_KEY:
    raise ValueError("❌ ANTHROPIC_API_KEY no encontrada en .env")

# Configuración de rutas
BASE_DIR = Path("storage/projects/conservacion_caminos")
BASES_DIR = BASE_DIR / "bases"
RESULTS_DIR = BASE_DIR / "results"
TEMP_DIR = BASE_DIR / "temp"

# Crear directorios si no existen
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
TEMP_DIR.mkdir(parents=True, exist_ok=True)

# Inicializar cliente Anthropic
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

print("✅ Configuración completa")
print(f"📁 Directorio bases: {BASES_DIR}")
print(f"📁 Directorio resultados: {RESULTS_DIR}")
print(f"📁 Directorio temporal: {TEMP_DIR}")
print(f"🔑 API Keys configuradas")
print(f"🚀 Listo para procesamiento paralelo")

✅ Configuración completa
📁 Directorio bases: storage/projects/conservacion_caminos/bases
📁 Directorio resultados: storage/projects/conservacion_caminos/results
📁 Directorio temporal: storage/projects/conservacion_caminos/temp
🔑 API Keys configuradas
🚀 Listo para procesamiento paralelo


In [2]:
# ============================================================================
# CELDA 1-B: AGREGAR IMPORTS FALTANTES (ejecutar después de CELDA 1)
# ============================================================================

# Imports adicionales necesarios
from tqdm.notebook import tqdm  # Para barras de progreso en Jupyter

# Configuración global que faltaba
CONFIG = {
    'CHUNK_SIZE': 15,  # Páginas por chunk
    'MAX_WORKERS': 4,   # Workers paralelos
    'OCR_TIMEOUT': 600, # Timeout para OCR (10 minutos)
    'MAX_TEXT_FOR_AI': 80000,  # Caracteres máximos para Claude
    'MIN_VALID_TEXT': 1000,    # Mínimo de caracteres para considerar válido
}

print("✅ Configuración adicional cargada")
print(f"   - tqdm importado para barras de progreso")
print(f"   - CONFIG definido con parámetros del sistema")

✅ Configuración adicional cargada
   - tqdm importado para barras de progreso
   - CONFIG definido con parámetros del sistema


In [3]:
# ============================================================================
# CELDA 2: CLASE PARA DIVISIÓN DE PDFs
# ============================================================================

class PDFSplitter:
    """Divide PDFs grandes en chunks para procesamiento paralelo."""
    
    def __init__(self, max_pages_per_chunk: int = 30):
        self.max_pages_per_chunk = max_pages_per_chunk
    
    def split_pdf(self, pdf_path: Path, output_dir: Path = None) -> List[Tuple[Path, int, int]]:
        """
        Divide PDF en chunks y retorna lista de (chunk_path, start_page, end_page).
        """
        if output_dir is None:
            output_dir = TEMP_DIR / f"{pdf_path.stem}_chunks"
        
        output_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"\n📄 Dividiendo PDF: {pdf_path.name}")
        print(f"   📏 Tamaño: {pdf_path.stat().st_size / 1024 / 1024:.1f} MB")
        
        # Obtener total de páginas
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            total_pages = len(pdf_reader.pages)
        
        print(f"   📑 Total páginas: {total_pages}")
        
        chunks_info = []
        chunk_number = 1
        
        for start_page in range(0, total_pages, self.max_pages_per_chunk):
            end_page = min(start_page + self.max_pages_per_chunk, total_pages)
            
            # Crear nombre del chunk
            chunk_filename = f"{pdf_path.stem}_chunk_{chunk_number:02d}_p{start_page+1}-{end_page}.pdf"
            chunk_path = output_dir / chunk_filename
            
            # Escribir chunk
            with open(pdf_path, 'rb') as input_file:
                pdf_reader = PyPDF2.PdfReader(input_file)
                pdf_writer = PyPDF2.PdfWriter()
                
                for page_num in range(start_page, end_page):
                    pdf_writer.add_page(pdf_reader.pages[page_num])
                
                with open(chunk_path, 'wb') as output_file:
                    pdf_writer.write(output_file)
            
            chunks_info.append((chunk_path, start_page + 1, end_page))
            print(f"   ✅ Chunk {chunk_number}: páginas {start_page+1}-{end_page}")
            chunk_number += 1
        
        print(f"   📦 Total chunks creados: {len(chunks_info)}")
        return chunks_info
    
    def cleanup_chunks(self, chunks_dir: Path):
        """Limpia los archivos temporales de chunks."""
        try:
            if chunks_dir.exists():
                for file in chunks_dir.glob("*.pdf"):
                    file.unlink()
                chunks_dir.rmdir()
                print(f"   🧹 Limpieza completada: {chunks_dir.name}")
        except Exception as e:
            print(f"   ⚠️ Error limpiando chunks: {e}")

# Inicializar splitter
splitter = PDFSplitter(max_pages_per_chunk=30)
print("✅ PDFSplitter inicializado (30 páginas por chunk)")

✅ PDFSplitter inicializado (30 páginas por chunk)


In [ ]:
# ============================================================================
# CELDA 3: FUNCIONES OCR MEJORADAS - PARALELO REAL Y MAYOR EXTRACCIÓN
# ============================================================================

def apply_ocr_enhanced(chunk_path: Path, api_key: str, 
                       timeout: int = 600, retry_count: int = 2) -> Dict[str, Any]:
    """
    OCR mejorado con reintentos y mejor extracción de texto.
    """
    result = {
        "chunk_name": chunk_path.name,
        "success": False,
        "text": "",
        "error": None,
        "processing_time": 0,
        "characters": 0,
        "retry_attempts": 0
    }
    
    start_time = time.time()
    
    for attempt in range(retry_count + 1):
        try:
            print(f"      🔄 Intento {attempt + 1} para {chunk_path.name}")
            
            # PASO 1: Aplicar OCR
            ocr_url = "https://api.pdfrest.com/pdf-with-ocr-text"
            
            with open(chunk_path, 'rb') as file:
                files = [('file', (chunk_path.name, file, 'application/pdf'))]
                headers = {'Api-Key': api_key}
                payload = {
                    'output': f'ocr_{chunk_path.stem}',
                    'languages': 'Spanish,English',  # Añadido inglés también
                    'ocr_library': 'tesseract'      # Especificar librería
                }
                
                response = requests.post(
                    ocr_url,
                    headers=headers,
                    data=payload,
                    files=files,
                    timeout=timeout
                )
            
            if response.status_code != 200:
                raise Exception(f"OCR failed: HTTP {response.status_code}")
            
            # Obtener URL del PDF procesado
            data = response.json()
            output_url = data.get('outputUrl')
            
            if not output_url:
                raise Exception("No output URL from OCR")
            
            # PASO 2: Descargar PDF procesado
            pdf_response = requests.get(output_url, timeout=120)
            if pdf_response.status_code != 200:
                raise Exception(f"Download failed: HTTP {pdf_response.status_code}")
            
            # PASO 3: Guardar temporalmente y extraer texto
            temp_pdf = TEMP_DIR / f"temp_ocr_{chunk_path.stem}_{attempt}.pdf"
            
            with open(temp_pdf, 'wb') as f:
                f.write(pdf_response.content)
            
            # PASO 4: Extraer texto con múltiples métodos
            extracted_text = ""
            
            # Método 1: API pdfRest para extracción
            extract_url = "https://api.pdfrest.com/extracted-text"
            with open(temp_pdf, 'rb') as file:
                files = [('file', (temp_pdf.name, file, 'application/pdf'))]
                headers = {'Api-Key': api_key}
                response = requests.post(extract_url, headers=headers, files=files, timeout=120)
                
                if response.status_code == 200:
                    extracted_text = response.json().get('fullText', '')
            
            # Método 2: Si falla o está vacío, usar PyPDF2 como respaldo
            if len(extracted_text) < 100:
                try:
                    with open(temp_pdf, 'rb') as f:
                        reader = PyPDF2.PdfReader(f)
                        backup_text = ""
                        for page in reader.pages:
                            backup_text += page.extract_text()
                        
                        if len(backup_text) > len(extracted_text):
                            extracted_text = backup_text
                            print(f"         📋 Usando PyPDF2 como respaldo ({len(backup_text)} chars)")
                except:
                    pass
            
            # Limpiar watermarks y texto basura
            extracted_text = re.sub(r'\[pdfRest.*?\]', '', extracted_text)
            extracted_text = re.sub(r'Page \d+ of \d+', '', extracted_text)
            extracted_text = re.sub(r'\n{3,}', '\n\n', extracted_text)
            
            # Limpiar archivo temporal
            if temp_pdf.exists():
                temp_pdf.unlink()
            
            if len(extracted_text) > 50:  # Mínimo 50 caracteres para considerarlo válido
                result["success"] = True
                result["text"] = extracted_text
                result["characters"] = len(extracted_text)
                result["retry_attempts"] = attempt
                print(f"      ✅ Éxito: {len(extracted_text):,} caracteres extraídos")
                break
            else:
                raise Exception(f"Texto insuficiente: solo {len(extracted_text)} caracteres")
                
        except requests.exceptions.Timeout:
            result["error"] = f"Timeout en intento {attempt + 1}"
            print(f"      ⏱️ Timeout en intento {attempt + 1}")
        except Exception as e:
            result["error"] = str(e)
            print(f"      ❌ Error en intento {attempt + 1}: {str(e)[:50]}")
        
        if attempt < retry_count:
            time.sleep(2)  # Esperar antes de reintentar
    
    result["processing_time"] = time.time() - start_time
    return result


def process_chunks_parallel_enhanced(chunks_info: List[Tuple[Path, int, int]], 
                                    api_key: str,
                                    max_workers: int = 4) -> Dict[str, Any]:
    """
    Procesamiento paralelo real y optimizado de chunks.
    """
    print(f"\n🚀 PROCESAMIENTO PARALELO OPTIMIZADO")
    print(f"   📦 Chunks a procesar: {len(chunks_info)}")
    print(f"   👷 Workers paralelos: {max_workers}")
    print("   " + "="*60)
    
    results = []
    all_texts = {}  # Diccionario para mantener orden de páginas
    successful = 0
    failed = 0
    total_chars = 0
    
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Crear futures para todos los chunks
        future_to_chunk = {
            executor.submit(apply_ocr_enhanced, chunk_path, api_key): (chunk_path, start, end)
            for chunk_path, start, end in chunks_info
        }
        
        # Procesar resultados conforme se completan
        with tqdm(total=len(chunks_info), desc="   Procesando chunks") as pbar:
            for future in as_completed(future_to_chunk):
                chunk_path, start_page, end_page = future_to_chunk[future]
                
                try:
                    result = future.result(timeout=700)  # Timeout para el future
                    results.append(result)
                    
                    if result["success"]:
                        all_texts[start_page] = {
                            'pages': (start_page, end_page),
                            'text': result['text'],
                            'chars': result['characters']
                        }
                        successful += 1
                        total_chars += result['characters']
                        print(f"   ✅ Páginas {start_page:3d}-{end_page:3d}: {result['characters']:,} chars")
                    else:
                        failed += 1
                        print(f"   ❌ Páginas {start_page:3d}-{end_page:3d}: {result['error'][:50]}")
                        
                except Exception as e:
                    failed += 1
                    print(f"   ❌ Páginas {start_page:3d}-{end_page:3d}: Error crítico: {str(e)[:50]}")
                    results.append({
                        "chunk_name": chunk_path.name,
                        "success": False,
                        "error": str(e)
                    })
                
                pbar.update(1)
    
    # Consolidar texto en orden correcto de páginas
    sorted_pages = sorted(all_texts.keys())
    consolidated_text = ""
    
    for page_num in sorted_pages:
        page_data = all_texts[page_num]
        consolidated_text += f"\n\n{'='*80}\n"
        consolidated_text += f"PÁGINAS {page_data['pages'][0]}-{page_data['pages'][1]}\n"
        consolidated_text += f"{'='*80}\n\n"
        consolidated_text += page_data['text']
    
    elapsed_time = time.time() - start_time
    
    # Estadísticas finales
    print(f"\n   📊 ESTADÍSTICAS DEL PROCESAMIENTO:")
    print(f"      ✅ Chunks exitosos: {successful}/{len(chunks_info)}")
    print(f"      ❌ Chunks fallidos: {failed}/{len(chunks_info)}")
    print(f"      📝 Total caracteres: {total_chars:,}")
    print(f"      ⏱️ Tiempo total: {elapsed_time:.1f}s")
    print(f"      ⚡ Velocidad: {elapsed_time/len(chunks_info):.1f}s por chunk")
    
    return {
        "success": successful > 0,
        "text": consolidated_text,
        "chunks_processed": len(chunks_info),
        "chunks_successful": successful,
        "chunks_failed": failed,
        "total_characters": len(consolidated_text),
        "total_processing_time": elapsed_time,
        "avg_time_per_chunk": elapsed_time / len(chunks_info) if chunks_info else 0,
        "results": results
    }

print("✅ Funciones OCR optimizadas cargadas")
print("   - Reintentos automáticos")
print("   - Procesamiento paralelo real")
print("   - Múltiples métodos de extracción")

In [ ]:
# ============================================================================
# CELDA 3-B: FUNCIÓN process_chunks_parallel ORIGINAL (que faltaba)
# ============================================================================

def process_chunks_parallel(chunks_info: List[Tuple[Path, int, int]], 
                          api_key: str,
                          max_workers: int = 3) -> Dict[str, Any]:
    """
    Procesa múltiples chunks en paralelo - versión compatible.
    """
    print(f"\n🚀 Procesando {len(chunks_info)} chunks en paralelo (max {max_workers} workers)")
    print("=" * 60)
    
    results = []
    all_texts = []
    successful = 0
    failed = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Enviar todos los chunks a procesar
        future_to_chunk = {
            executor.submit(apply_ocr_to_chunk, chunk_path, api_key): (chunk_path, start, end)
            for chunk_path, start, end in chunks_info
        }
        
        # Procesar resultados conforme se completan
        for future in as_completed(future_to_chunk):
            chunk_path, start_page, end_page = future_to_chunk[future]
            
            try:
                result = future.result()
                results.append(result)
                
                if result["success"]:
                    all_texts.append(f"\n--- Páginas {start_page}-{end_page} ---\n{result['text']}")
                    successful += 1
                    print(f"   ✅ {result['chunk_name']}: {result['characters']:,} chars en {result['processing_time']:.1f}s")
                else:
                    failed += 1
                    print(f"   ❌ {result['chunk_name']}: {result['error']}")
                    
            except Exception as e:
                failed += 1
                print(f"   ❌ {chunk_path.name}: Error inesperado: {e}")
                results.append({
                    "chunk_name": chunk_path.name,
                    "success": False,
                    "error": str(e)
                })
    
    # Consolidar texto
    consolidated_text = "\n".join(all_texts)
    
    total_time = sum(r.get("processing_time", 0) for r in results)
    
    return {
        "success": successful > 0,
        "text": consolidated_text,
        "chunks_processed": len(chunks_info),
        "chunks_successful": successful,
        "chunks_failed": failed,
        "total_characters": len(consolidated_text),
        "total_processing_time": total_time,
        "results": results
    }

print("✅ process_chunks_parallel() definida")

In [ ]:
# ============================================================================
# CELDA 3-C: FUNCIÓN apply_ocr_to_chunk ORIGINAL (que faltaba)
# ============================================================================

def apply_ocr_to_chunk(chunk_path: Path, api_key: str, timeout: int = 300) -> Dict[str, Any]:
    """
    Aplica OCR a un chunk individual usando pdfRest.
    """
    result = {
        "chunk_name": chunk_path.name,
        "success": False,
        "text": "",
        "error": None,
        "processing_time": 0,
        "characters": 0
    }
    
    start_time = time.time()
    
    try:
        # Paso 1: Aplicar OCR al PDF
        ocr_url = "https://api.pdfrest.com/pdf-with-ocr-text"
        
        with open(chunk_path, 'rb') as file:
            payload = {
                'output': f'ocr_{chunk_path.stem}',
                'languages': 'Spanish'
            }
            files = [('file', (chunk_path.name, file, 'application/pdf'))]
            headers = {'Api-Key': api_key}
            
            response = requests.post(
                ocr_url,
                headers=headers,
                data=payload,
                files=files,
                timeout=timeout
            )
        
        if response.status_code != 200:
            result["error"] = f"OCR failed: HTTP {response.status_code}"
            return result
        
        # Obtener URL del PDF procesado
        data = response.json()
        output_url = data.get('outputUrl')
        
        if not output_url:
            result["error"] = "No output URL from OCR"
            return result
        
        # Paso 2: Descargar PDF procesado
        pdf_response = requests.get(output_url, timeout=60)
        if pdf_response.status_code != 200:
            result["error"] = f"Download failed: HTTP {pdf_response.status_code}"
            return result
        
        # Paso 3: Extraer texto del PDF procesado
        temp_pdf = TEMP_DIR / f"temp_ocr_{chunk_path.name}"
        try:
            with open(temp_pdf, 'wb') as f:
                f.write(pdf_response.content)
            
            # Extraer texto
            extract_url = "https://api.pdfrest.com/extracted-text"
            
            with open(temp_pdf, 'rb') as file:
                files = [('file', (temp_pdf.name, file, 'application/pdf'))]
                headers = {'Api-Key': api_key}
                
                response = requests.post(extract_url, headers=headers, files=files, timeout=60)
            
            if response.status_code == 200:
                text = response.json().get('fullText', '')
                # Limpiar watermarks
                text = re.sub(r'\[pdfRest Free Demo\]', '', text)
                
                result["success"] = True
                result["text"] = text
                result["characters"] = len(text)
            else:
                result["error"] = f"Text extraction failed: HTTP {response.status_code}"
                
        finally:
            if temp_pdf.exists():
                temp_pdf.unlink()
        
    except requests.exceptions.Timeout:
        result["error"] = f"Timeout después de {timeout}s"
    except Exception as e:
        result["error"] = str(e)
    
    result["processing_time"] = time.time() - start_time
    return result

print("✅ apply_ocr_to_chunk() definida")

In [4]:
# ============================================================================
# CELDA 4: FUNCIONES DE EXTRACCIÓN DE PATRONES
# ============================================================================

def extract_patterns_from_text(text: str) -> Dict[str, List]:
    """
    Extrae patrones específicos del texto usando regex.
    """
    patterns = {
        # Códigos MOP estándar
        'mop_codes': list(set(re.findall(r'7\.\d{3}\.\d+[a-z]*\d*', text, re.IGNORECASE))),
        
        # Códigos ETE
        'ete_codes': list(set(re.findall(r'ETE[\.\-\s]?\d+', text, re.IGNORECASE))),
        
        # Otros códigos
        'other_codes': list(set(re.findall(r'804[\-\.]?\d+', text, re.IGNORECASE))),
        
        # Códigos SAFI
        'safi_codes': list(set(re.findall(r'SAFI\s*[:\s]*(\d+)', text, re.IGNORECASE))),
        
        # Montos en pesos chilenos
        'montos': re.findall(r'\$\s*[\d\.,]+', text),
        
        # Cantidades con unidades
        'cantidades': re.findall(r'(\d+[\.,]?\d*)\s*(km|m3|m2|m|ton|kg|gl|un|lt|há)', text, re.IGNORECASE),
        
        # Porcentajes
        'porcentajes': re.findall(r'\d+[\.,]?\d*\s*%', text)
    }
    
    # Ordenar códigos
    for key in ['mop_codes', 'ete_codes', 'other_codes', 'safi_codes']:
        patterns[key].sort()
    
    return patterns

print("✅ Función de extracción de patrones cargada")

✅ Función de extracción de patrones cargada


In [ ]:
# ============================================================================
# CELDA 5: ANÁLISIS CON CLAUDE HAIKU
# ============================================================================

def advanced_haiku_analysis(text: str, filename: str, max_chars: int = 50000) -> Dict:
    """
    Análisis detallado usando Claude 3.5 Haiku.
    """
    print(f"🤖 Analizando con Claude 3.5 Haiku: {filename}")
    
    # Limitar texto para análisis
    text_to_analyze = text[:max_chars] if len(text) > max_chars else text
    
    prompt = f"""Analiza este documento técnico MOP chileno y extrae información estructurada.

DOCUMENTO: {filename}
TEXTO (primeros {len(text_to_analyze)} caracteres):
{text_to_analyze}

Extrae la siguiente información en formato JSON:
{{
  "proyecto": {{
    "nombre": "nombre completo del proyecto",
    "codigo_safi": "código SAFI si existe",
    "tipo_obra": "tipo de obra",
    "ubicacion": {{
      "region": "región",
      "provincia": "provincia",
      "comunas": ["lista de comunas"],
      "sectores": ["sectores específicos"]
    }}
  }},
  "presupuesto": {{
    "total_estimado": "monto total si se menciona",
    "items_principales": [
      {{
        "codigo": "código MOP o ETE",
        "descripcion": "descripción del item",
        "unidad": "unidad de medida",
        "cantidad": "cantidad",
        "precio_unitario": "precio unitario",
        "total": "total del item"
      }}
    ]
  }},
  "codigos": {{
    "mop": ["lista de códigos MOP encontrados"],
    "ete": ["lista de códigos ETE"],
    "otros": ["otros códigos"]
  }},
  "especificaciones": {{
    "materiales": ["principales materiales"],
    "equipos": ["equipos mencionados"],
    "normativas": ["normas técnicas aplicables"]
  }},
  "plazos": {{
    "total_dias": "plazo total en días",
    "inicio": "fecha de inicio si se menciona",
    "termino": "fecha de término si se menciona"
  }}
}}

Responde SOLO con el JSON, sin texto adicional."""

    try:
        response = client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=4000,
            temperature=0,
            messages=[{"role": "user", "content": prompt}]
        )
        
        response_text = response.content[0].text
        
        # Intentar parsear el JSON
        try:
            json_start = response_text.find('{')
            json_end = response_text.rfind('}') + 1
            if json_start >= 0 and json_end > json_start:
                json_text = response_text[json_start:json_end]
                analysis = json.loads(json_text)
            else:
                analysis = {"raw_response": response_text}
        except json.JSONDecodeError:
            analysis = {"raw_response": response_text}
        
        return {
            "success": True,
            "analysis": analysis,
            "model_used": "claude-3.5-haiku-20241022"
        }
        
    except Exception as e:
        print(f"   ❌ Error en análisis: {e}")
        return {
            "success": False,
            "error": str(e)
        }

print("✅ Función de análisis con Claude cargada")

In [ ]:
# ============================================================================
# CELDA 6: FUNCIÓN DE PROCESAMIENTO OCR COMPLETO
# ============================================================================

def process_pdf_ocr(pdf_path: Path, use_parallel: bool = True, max_workers: int = 3) -> Dict[str, Any]:
    """
    Procesa un PDF completo con OCR: división, OCR paralelo, guardado de texto.
    """
    print(f"\n{'='*70}")
    print(f"📄 PROCESAMIENTO OCR: {pdf_path.name}")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    # Verificar que existe
    if not pdf_path.exists():
        return {"success": False, "error": f"Archivo no encontrado: {pdf_path}"}
    
    # Verificar si ya existe el texto extraído
    text_file = RESULTS_DIR / f"{pdf_path.stem}_texto.txt"
    if text_file.exists():
        print(f"✅ Texto ya extraído previamente: {text_file.name}")
        with open(text_file, 'r', encoding='utf-8') as f:
            text = f.read()
        return {
            "success": True,
            "text": text,
            "cached": True,
            "text_file": str(text_file),
            "total_characters": len(text)
        }
    
    # Paso 1: Dividir PDF
    chunks_dir = TEMP_DIR / f"{pdf_path.stem}_chunks"
    chunks_info = splitter.split_pdf(pdf_path, chunks_dir)
    
    # Paso 2: Procesar chunks (paralelo o secuencial)
    if use_parallel:
        ocr_result = process_chunks_parallel(chunks_info, PDF_REST_API_KEY, max_workers)
    else:
        # Procesamiento secuencial (fallback)
        all_texts = []
        for chunk_path, start, end in chunks_info:
            print(f"   📋 Procesando páginas {start}-{end}...")
            result = apply_ocr_to_chunk(chunk_path, PDF_REST_API_KEY)
            if result["success"]:
                all_texts.append(result["text"])
                print(f"      ✅ {result['characters']:,} caracteres")
        
        ocr_result = {
            "success": len(all_texts) > 0,
            "text": "\n".join(all_texts),
            "chunks_processed": len(chunks_info),
            "chunks_successful": len(all_texts)
        }
    
    if not ocr_result["success"]:
        splitter.cleanup_chunks(chunks_dir)
        return {"success": False, "error": "Falló la extracción OCR"}
    
    text = ocr_result["text"]
    
    # Paso 3: Guardar texto extraído
    with open(text_file, 'w', encoding='utf-8') as f:
        f.write(text)
    print(f"   💾 Texto guardado: {text_file.name}")
    
    # Paso 4: Limpiar chunks temporales
    splitter.cleanup_chunks(chunks_dir)
    
    # Mostrar resumen
    total_time = time.time() - start_time
    print(f"\n✅ OCR COMPLETADO")
    print(f"   ⏱️ Tiempo total: {total_time:.1f}s")
    print(f"   📝 Caracteres extraídos: {len(text):,}")
    print(f"   💾 Archivo: {text_file}")
    
    return {
        "success": True,
        "text": text,
        "cached": False,
        "text_file": str(text_file),
        "total_characters": len(text),
        "processing_time": total_time,
        "chunks_processed": ocr_result["chunks_processed"],
        "chunks_successful": ocr_result["chunks_successful"]
    }

print("✅ Función de procesamiento OCR cargada")

In [ ]:
# ============================================================================
# CELDA 6-B: CORREGIR PROBLEMA DE CODIFICACIÓN EN GUARDADO
# ============================================================================

def save_text_safe(text: str, filepath: Path) -> bool:
    """
    Guarda texto con manejo seguro de caracteres problemáticos.
    """
    try:
        # Método 1: Intentar guardar normalmente
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(text)
        return True
    except UnicodeEncodeError:
        try:
            # Método 2: Limpiar caracteres problemáticos
            cleaned_text = text.encode('utf-8', errors='ignore').decode('utf-8')
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(cleaned_text)
            print(f"   ⚠️ Se limpiaron caracteres problemáticos del texto")
            return True
        except Exception as e:
            print(f"   ❌ Error guardando texto: {e}")
            return False

print("✅ save_text_safe() definida para manejar problemas de codificación")

In [ ]:
# ============================================================================
# CELDA 6-C: VERSIÓN CORREGIDA DE process_pdf_ocr
# ============================================================================

def process_pdf_ocr(pdf_path: Path, use_parallel: bool = True, max_workers: int = 3) -> Dict[str, Any]:
    """
    Procesa un PDF completo con OCR - VERSIÓN CORREGIDA.
    """
    print(f"\n{'='*70}")
    print(f"📄 PROCESAMIENTO OCR: {pdf_path.name}")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    # Verificar que existe
    if not pdf_path.exists():
        return {"success": False, "error": f"Archivo no encontrado: {pdf_path}"}
    
    # Verificar si ya existe el texto extraído
    text_file = RESULTS_DIR / f"{pdf_path.stem}_texto.txt"
    if text_file.exists():
        print(f"✅ Texto ya extraído previamente: {text_file.name}")
        try:
            with open(text_file, 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read()
            return {
                "success": True,
                "text": text,
                "cached": True,
                "text_file": str(text_file),
                "total_characters": len(text)
            }
        except Exception as e:
            print(f"   ⚠️ Error leyendo archivo existente: {e}")
    
    # Paso 1: Dividir PDF
    chunks_dir = TEMP_DIR / f"{pdf_path.stem}_chunks"
    chunks_info = splitter.split_pdf(pdf_path, chunks_dir)
    
    if not chunks_info:
        return {"success": False, "error": "No se pudieron crear chunks del PDF"}
    
    # Paso 2: Procesar chunks
    if use_parallel:
        ocr_result = process_chunks_parallel(chunks_info, PDF_REST_API_KEY, max_workers)
    else:
        # Procesamiento secuencial
        all_texts = []
        for chunk_path, start, end in chunks_info:
            print(f"   📋 Procesando páginas {start}-{end}...")
            result = apply_ocr_to_chunk(chunk_path, PDF_REST_API_KEY)
            if result["success"]:
                all_texts.append(result["text"])
                print(f"      ✅ {result['characters']:,} caracteres")
        
        ocr_result = {
            "success": len(all_texts) > 0,
            "text": "\n".join(all_texts),
            "chunks_processed": len(chunks_info),
            "chunks_successful": len(all_texts)
        }
    
    if not ocr_result["success"]:
        splitter.cleanup_chunks(chunks_dir)
        return {"success": False, "error": "Falló la extracción OCR"}
    
    text = ocr_result["text"]
    
    # Paso 3: Guardar texto extraído con manejo seguro
    if save_text_safe(text, text_file):
        print(f"   💾 Texto guardado: {text_file.name}")
    else:
        print(f"   ⚠️ No se pudo guardar el texto, pero continuando...")
    
    # Paso 4: Limpiar chunks temporales
    splitter.cleanup_chunks(chunks_dir)
    
    # Mostrar resumen
    total_time = time.time() - start_time
    print(f"\n✅ OCR COMPLETADO")
    print(f"   ⏱️ Tiempo total: {total_time:.1f}s")
    print(f"   📝 Caracteres extraídos: {len(text):,}")
    print(f"   💾 Archivo: {text_file}")
    
    return {
        "success": True,
        "text": text,
        "cached": False,
        "text_file": str(text_file),
        "total_characters": len(text),
        "processing_time": total_time,
        "chunks_processed": ocr_result["chunks_processed"],
        "chunks_successful": ocr_result["chunks_successful"]
    }

print("✅ process_pdf_ocr() corregida con manejo seguro de codificación")

# ============================================================================
# VERIFICACIÓN FINAL
# ============================================================================

print("\n" + "="*80)
print("✅ TODAS LAS FUNCIONES NECESARIAS HAN SIDO DEFINIDAS")
print("="*80)
print("\nFunciones disponibles:")
print("  • apply_ocr_to_chunk() - OCR individual")
print("  • process_chunks_parallel() - Procesamiento paralelo")
print("  • process_pdf_ocr() - Procesamiento completo con manejo de errores")
print("  • save_text_safe() - Guardado seguro de texto")
print("  • CONFIG - Configuración global del sistema")
print("\nAhora puedes ejecutar:")
print("  >>> results = process_all_bases_complete()")
print("="*80)

In [ ]:
# ============================================================================
# CELDA 7: FUNCIÓN DE ANÁLISIS COMPLETO
# ============================================================================

def analyze_pdf_document(pdf_path: Path, force_ocr: bool = False) -> Dict[str, Any]:
    """
    Análisis completo de un documento PDF: OCR + Extracción + Análisis IA.
    """
    print(f"\n{'='*80}")
    print(f"📚 ANÁLISIS COMPLETO: {pdf_path.name}")
    print(f"{'='*80}")
    
    if not pdf_path.exists():
        print(f"❌ Archivo no encontrado: {pdf_path}")
        return {"success": False, "error": "Archivo no encontrado"}
    
    # PASO 1: OCR - Extraer texto
    print(f"\n📝 PASO 1: Extracción de texto...")
    ocr_result = process_pdf_ocr(pdf_path, use_parallel=True, max_workers=3)
    
    if not ocr_result["success"]:
        return ocr_result
    
    text = ocr_result["text"]
    
    # Verificar si hay texto extraído
    if not text or len(text) < 100:
        print(f"⚠️ Texto extraído muy corto ({len(text)} caracteres)")
        print("   Posible problema con el OCR o PDF sin texto")
    
    # PASO 2: Extracción de patrones
    print(f"\n📊 PASO 2: Extrayendo patrones...")
    patterns = extract_patterns_from_text(text)
    
    print(f"   ✅ Códigos MOP: {len(patterns['mop_codes'])}")
    print(f"   ✅ Códigos ETE: {len(patterns['ete_codes'])}")
    print(f"   ✅ Códigos SAFI: {len(patterns['safi_codes'])}")
    print(f"   ✅ Montos encontrados: {len(patterns['montos'])}")
    
    # PASO 3: Análisis con Claude
    print(f"\n🤖 PASO 3: Análisis con IA...")
    ai_analysis = advanced_haiku_analysis(text, pdf_path.name)
    
    if ai_analysis['success']:
        print(f"   ✅ Análisis completado")
    else:
        print(f"   ⚠️ Error en análisis: {ai_analysis.get('error')}")
    
    # PASO 4: Consolidar resultados
    final_result = {
        "success": True,
        "filename": pdf_path.name,
        "file_path": str(pdf_path),
        "timestamp": datetime.now().isoformat(),
        "extraction": {
            "text_file": ocr_result["text_file"],
            "total_characters": len(text),
            "total_lines": len(text.split('\n')),
            "total_words": len(text.split()),
            "cached": ocr_result.get("cached", False)
        },
        "patterns_extracted": patterns,
        "ai_analysis": ai_analysis,
        "summary": {}
    }
    
    # Crear resumen ejecutivo
    if ai_analysis.get('success') and isinstance(ai_analysis.get('analysis'), dict):
        analysis = ai_analysis['analysis']
        
        # Acceso seguro a estructuras anidadas
        proyecto = analysis.get('proyecto', {}) or {}
        presupuesto = analysis.get('presupuesto', {}) or {}
        ubicacion = proyecto.get('ubicacion', {}) or {}
        
        final_result['summary'] = {
            'proyecto': proyecto.get('nombre', 'No identificado') or 'No identificado',
            'region': ubicacion.get('region', 'N/D') or 'N/D',
            'comunas': ubicacion.get('comunas', []) or [],
            'tipo_obra': proyecto.get('tipo_obra', 'N/D') or 'N/D',
            'presupuesto_estimado': presupuesto.get('total_estimado', 'N/D') or 'N/D',
            'items_principales': len(presupuesto.get('items_principales', []) or []),
            'codigos_mop': len(patterns['mop_codes']),
            'codigos_ete': len(patterns['ete_codes']),
            'codigos_safi': len(patterns['safi_codes'])
        }
    else:
        # Si no hay análisis o falló, crear resumen básico
        final_result['summary'] = {
            'proyecto': 'No identificado',
            'region': 'N/D',
            'comunas': [],
            'tipo_obra': 'N/D',
            'presupuesto_estimado': 'N/D',
            'items_principales': 0,
            'codigos_mop': len(patterns['mop_codes']),
            'codigos_ete': len(patterns['ete_codes']),
            'codigos_safi': len(patterns['safi_codes'])
        }
    
    # PASO 5: Guardar resultados
    analysis_file = RESULTS_DIR / f"{pdf_path.stem}_analisis_completo.json"
    with open(analysis_file, 'w', encoding='utf-8') as f:
        json.dump(final_result, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ ANÁLISIS COMPLETADO")
    print(f"   💾 Resultados guardados: {analysis_file.name}")
    
    # Mostrar resumen
    s = final_result['summary']
    print(f"\n📋 RESUMEN:")
    print(f"   Proyecto: {s['proyecto'][:60]}")
    print(f"   Ubicación: {s['region']}, {', '.join(s['comunas'][:3]) if s['comunas'] else 'N/D'}")
    print(f"   Tipo obra: {s['tipo_obra']}")
    print(f"   Presupuesto: {s['presupuesto_estimado']}")
    print(f"   Items: {s['items_principales']}")
    print(f"   Códigos: {s['codigos_mop']} MOP, {s['codigos_ete']} ETE, {s['codigos_safi']} SAFI")
    
    return final_result

print("✅ Función de análisis completo cargada")

In [ ]:
# ============================================================================
# CELDA 8: PROCESAR TODOS LOS ARCHIVOS - CORREGIDO
# ============================================================================

def process_all_bases_complete():
    """
    Procesa TODOS los PDFs base con análisis completo.
    """
    print("\n" + "="*80)
    print("🚀 PROCESAMIENTO COMPLETO DE DOCUMENTOS MOP")
    print("="*80)
    
    # Lista de TODOS los PDFs a procesar
    pdf_files = []
    
    # Buscar todos los PDFs en el directorio bases
    for pdf_file in BASES_DIR.glob("*.pdf"):
        pdf_files.append(pdf_file)
    
    if not pdf_files:
        print("❌ No se encontraron archivos PDF en el directorio bases")
        return []
    
    print(f"\n📚 Archivos encontrados: {len(pdf_files)}")
    for pdf in pdf_files:
        print(f"   - {pdf.name} ({pdf.stat().st_size / 1024 / 1024:.1f} MB)")
    
    # Procesar cada archivo
    all_results = []
    summary_data = []
    
    for idx, pdf_path in enumerate(pdf_files, 1):
        print(f"\n{'='*80}")
        print(f"📄 [{idx}/{len(pdf_files)}] Procesando: {pdf_path.name}")
        print(f"{'='*80}")
        
        try:
            # Analizar el documento
            result = analyze_pdf_document(pdf_path, force_ocr=False)
            all_results.append(result)
            
            # Agregar al resumen
            if result.get('success'):
                summary_data.append({
                    'Archivo': pdf_path.name,
                    'Tamaño (MB)': round(pdf_path.stat().st_size / 1024 / 1024, 2),
                    'Caracteres': result['extraction']['total_characters'],
                    'Palabras': result['extraction']['total_words'],
                    'Códigos MOP': result['summary'].get('codigos_mop', 0),
                    'Códigos ETE': result['summary'].get('codigos_ete', 0),
                    'Códigos SAFI': result['summary'].get('codigos_safi', 0),
                    'Items': result['summary'].get('items_principales', 0),
                    'Región': result['summary'].get('region', 'N/D'),
                    'Tipo Obra': result['summary'].get('tipo_obra', 'N/D')[:30],
                    'Estado': '✅ Completado'
                })
            else:
                summary_data.append({
                    'Archivo': pdf_path.name,
                    'Tamaño (MB)': round(pdf_path.stat().st_size / 1024 / 1024, 2),
                    'Estado': f'❌ Error: {result.get("error", "Desconocido")}'
                })
                
        except Exception as e:
            print(f"❌ Error procesando {pdf_path.name}: {e}")
            summary_data.append({
                'Archivo': pdf_path.name,
                'Estado': f'❌ Excepción: {str(e)[:50]}'
            })
    
    # Mostrar resumen consolidado
    if summary_data:
        print(f"\n{'='*80}")
        print("📊 RESUMEN CONSOLIDADO")
        print("="*80)
        
        df_summary = pd.DataFrame(summary_data)
        display(df_summary)
        
        # Estadísticas globales
        successful = len([s for s in summary_data if '✅' in s.get('Estado', '')])
        failed = len(summary_data) - successful
        
        print(f"\n📈 ESTADÍSTICAS TOTALES:")
        print(f"   📄 Documentos procesados: {successful}/{len(pdf_files)}")
        print(f"   ❌ Documentos con errores: {failed}")
        
        if 'Caracteres' in df_summary.columns:
            total_chars = df_summary['Caracteres'].fillna(0).sum()
            avg_chars = df_summary['Caracteres'].fillna(0).mean()
            print(f"   📝 Total caracteres: {int(total_chars):,}")
            print(f"   📊 Promedio caracteres: {int(avg_chars):,}")
        
        if 'Códigos MOP' in df_summary.columns:
            total_mop = df_summary['Códigos MOP'].fillna(0).sum()
            print(f"   🔢 Total códigos MOP: {int(total_mop)}")
        
        # Guardar resumen en Excel
        excel_path = RESULTS_DIR / f"resumen_consolidado_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df_summary.to_excel(excel_path, index=False)
        print(f"\n💾 Resumen guardado en: {excel_path}")
    
    return all_results

print("✅ Función process_all_bases_complete() cargada")
print("   Esta función SÍ procesa TODOS los archivos PDF del directorio")

In [ ]:
# ============================================================================
# CELDA 9: EJECUTAR PROCESAMIENTO (EJECUTAR ESTA CELDA PARA INICIAR)
# ============================================================================

results = process_all_bases_complete()

In [ ]:
# ============================================================================
# CELDA 10: VERIFICAR ARCHIVOS EXISTENTES
# ============================================================================

def check_existing_files():
    """
    Verifica qué archivos ya han sido procesados y cuáles faltan.
    """
    print("📁 VERIFICACIÓN DE ARCHIVOS")
    print("="*80)
    
    # Archivos PDF base
    pdf_files = ["bases1.pdf", "bases2.pdf", "bases3.pdf"]
    
    for pdf_name in pdf_files:
        pdf_path = BASES_DIR / pdf_name
        text_file = RESULTS_DIR / f"{pdf_path.stem}_texto.txt"
        analysis_file = RESULTS_DIR / f"{pdf_path.stem}_analisis_completo.json"
        
        print(f"\n📄 {pdf_name}:")
        
        # Verificar PDF original
        if pdf_path.exists():
            size_mb = pdf_path.stat().st_size / 1024 / 1024
            print(f"   ✅ PDF existe ({size_mb:.1f} MB)")
        else:
            print(f"   ❌ PDF NO encontrado")
            continue
        
        # Verificar texto extraído
        if text_file.exists():
            with open(text_file, 'r', encoding='utf-8') as f:
                text_len = len(f.read())
            print(f"   ✅ Texto extraído ({text_len:,} caracteres)")
        else:
            print(f"   ⚠️ Texto NO extraído - requiere OCR")
        
        # Verificar análisis
        if analysis_file.exists():
            print(f"   ✅ Análisis completo existe")
            with open(analysis_file, 'r', encoding='utf-8') as f:
                analysis = json.load(f)
                if 'summary' in analysis:
                    s = analysis['summary']
                    print(f"      - Proyecto: {s.get('proyecto', 'N/D')[:50]}")
                    print(f"      - Códigos MOP: {s.get('codigos_mop', 0)}")
        else:
            print(f"   ⚠️ Análisis NO realizado")
    
    print("\n" + "="*80)
    return True

# Ejecutar verificación
check_existing_files()

In [ ]:
# ============================================================================
# CELDA 11: REPROCESAR ARCHIVO ESPECÍFICO - MEJORADO
# ============================================================================

def reprocess_file_enhanced(filename: str, force_ocr: bool = True, 
                           max_workers: int = 4, chunk_size: int = 15):
    """
    Reprocesa un archivo con configuración personalizada.
    
    Args:
        filename: Nombre del archivo o path completo
        force_ocr: Forzar nuevo OCR
        max_workers: Número de workers paralelos
        chunk_size: Páginas por chunk
    """
    # Buscar archivo
    if Path(filename).exists():
        pdf_path = Path(filename)
    else:
        pdf_path = BASES_DIR / filename
    
    if not pdf_path.exists():
        print(f"❌ Archivo no encontrado: {filename}")
        print(f"   Buscado en: {pdf_path}")
        return None
    
    print(f"\n🔄 REPROCESAMIENTO AVANZADO: {pdf_path.name}")
    print(f"   ⚙️ Configuración:")
    print(f"      - Workers: {max_workers}")
    print(f"      - Chunk size: {chunk_size} páginas")
    print(f"      - Forzar OCR: {force_ocr}")
    
    # Ajustar configuración temporal
    old_workers = CONFIG['MAX_WORKERS']
    old_chunk = CONFIG['CHUNK_SIZE']
    
    CONFIG['MAX_WORKERS'] = max_workers
    CONFIG['CHUNK_SIZE'] = chunk_size
    
    if force_ocr:
        # Eliminar archivos existentes
        text_file = RESULTS_DIR / f"{pdf_path.stem}_texto.txt"
        if text_file.exists():
            text_file.unlink()
            print(f"   🗑️ Archivo de texto anterior eliminado")
    
    # Procesar
    result = analyze_pdf_document(pdf_path, force_ocr=force_ocr)
    
    # Restaurar configuración
    CONFIG['MAX_WORKERS'] = old_workers
    CONFIG['CHUNK_SIZE'] = old_chunk
    
    if result['success']:
        print(f"\n✅ Reprocesamiento exitoso:")
        print(f"   📝 Caracteres extraídos: {result['extraction']['total_characters']:,}")
        print(f"   🔢 Códigos MOP: {result['summary']['codigos_mop']}")
    else:
        print(f"\n❌ Reprocesamiento fallido: {result.get('error')}")
    
    return result

print("✅ Función reprocess_file_enhanced() cargada")
print("   Ejemplo: reprocess_file_enhanced('bases3.pdf', max_workers=6, chunk_size=10)")

In [ ]:
# ============================================================================
# CELDA 12: GENERAR INFORME HTML CONSOLIDADO - COMPLETO
# ============================================================================

def generate_consolidated_report():
    """
    Genera un informe HTML consolidado de todos los análisis.
    """
    print("\n📊 GENERANDO INFORME CONSOLIDADO HTML")
    print("="*80)
    
    # Recopilar todos los análisis
    analysis_files = list(RESULTS_DIR.glob("*_analisis_completo.json"))
    
    if not analysis_files:
        print("⚠️ No se encontraron análisis para consolidar")
        return None
    
    print(f"📄 Archivos de análisis encontrados: {len(analysis_files)}")
    
    all_data = []
    for file in analysis_files:
        print(f"   - Cargando: {file.name}")
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_data.append(data)
    
    # Estadísticas globales
    total_chars = sum(data['extraction']['total_characters'] for data in all_data)
    total_mop = sum(data['summary'].get('codigos_mop', 0) for data in all_data)
    total_items = sum(data['summary'].get('items_principales', 0) for data in all_data)
    total_pages = sum(data['extraction'].get('total_pages', 0) for data in all_data)
    
    # Crear HTML completo
    html_content = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Informe Consolidado MOP - Análisis de Presupuestos</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: 'Segoe UI', -apple-system, BlinkMacSystemFont, Arial, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
            line-height: 1.6;
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }}
        
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 40px;
            text-align: center;
        }}
        
        .header h1 {{
            font-size: 2.8em;
            margin-bottom: 15px;
            font-weight: 300;
            text-shadow: 0 2px 4px rgba(0,0,0,0.3);
        }}
        
        .header .subtitle {{
            font-size: 1.3em;
            opacity: 0.9;
            margin-bottom: 20px;
        }}
        
        .header .date {{
            font-size: 1em;
            opacity: 0.8;
            background: rgba(255,255,255,0.1);
            padding: 10px 20px;
            border-radius: 25px;
            display: inline-block;
        }}
        
        .content {{
            padding: 40px;
        }}
        
        .summary-section {{
            margin-bottom: 50px;
        }}
        
        .section-title {{
            font-size: 2em;
            color: #333;
            margin-bottom: 30px;
            text-align: center;
            position: relative;
        }}
        
        .section-title::after {{
            content: '';
            position: absolute;
            bottom: -10px;
            left: 50%;
            transform: translateX(-50%);
            width: 60px;
            height: 3px;
            background: linear-gradient(135deg, #667eea, #764ba2);
            border-radius: 2px;
        }}
        
        .summary-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
            gap: 25px;
            margin: 40px 0;
        }}
        
        .summary-card {{
            background: white;
            border-radius: 15px;
            padding: 30px;
            text-align: center;
            box-shadow: 0 8px 25px rgba(0,0,0,0.1);
            transition: all 0.3s ease;
            border: 2px solid transparent;
            position: relative;
            overflow: hidden;
        }}
        
        .summary-card::before {{
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            height: 4px;
            background: linear-gradient(135deg, #667eea, #764ba2);
        }}
        
        .summary-card:hover {{
            transform: translateY(-5px);
            box-shadow: 0 15px 35px rgba(0,0,0,0.15);
        }}
        
        .summary-value {{
            font-size: 2.5em;
            font-weight: 700;
            background: linear-gradient(135deg, #667eea, #764ba2);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
            margin-bottom: 10px;
        }}
        
        .summary-label {{
            color: #666;
            font-size: 1.1em;
            font-weight: 500;
            text-transform: uppercase;
            letter-spacing: 1px;
        }}
        
        .documents-table {{
            width: 100%;
            background: white;
            border-radius: 15px;
            overflow: hidden;
            box-shadow: 0 8px 25px rgba(0,0,0,0.1);
            margin-top: 30px;
        }}
        
        .documents-table th {{
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
            padding: 20px 15px;
            text-align: left;
            font-weight: 600;
            text-transform: uppercase;
            letter-spacing: 0.5px;
            font-size: 0.9em;
        }}
        
        .documents-table td {{
            padding: 18px 15px;
            border-bottom: 1px solid #eee;
            vertical-align: top;
        }}
        
        .documents-table tr:last-child td {{
            border-bottom: none;
        }}
        
        .documents-table tbody tr:hover {{
            background: #f8f9ff;
        }}
        
        .doc-name {{
            font-weight: 600;
            color: #333;
            font-size: 1em;
        }}
        
        .project-name {{
            color: #555;
            max-width: 300px;
            word-wrap: break-word;
        }}
        
        .no-data {{
            color: #999;
            font-style: italic;
        }}
        
        .mop-count {{
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
            padding: 5px 12px;
            border-radius: 20px;
            font-size: 0.9em;
            font-weight: 600;
            display: inline-block;
        }}
        
        .items-count {{
            background: #28a745;
            color: white;
            padding: 5px 12px;
            border-radius: 20px;
            font-size: 0.9em;
            font-weight: 600;
            display: inline-block;
        }}
        
        .chars-count {{
            color: #666;
            font-size: 0.9em;
        }}
        
        .location {{
            color: #555;
            font-size: 0.95em;
        }}
        
        .footer {{
            text-align: center;
            padding: 30px;
            background: #f8f9fa;
            color: #666;
            font-size: 0.9em;
        }}
        
        @media (max-width: 768px) {{
            .header h1 {{ font-size: 2em; }}
            .summary-grid {{ grid-template-columns: 1fr; }}
            .content {{ padding: 20px; }}
            .documents-table {{ font-size: 0.9em; }}
            .documents-table th, .documents-table td {{ padding: 12px 8px; }}
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>📊 Informe Consolidado MOP</h1>
            <div class="subtitle">Análisis de Bases de Licitación y Presupuestos</div>
            <div class="date">📅 Generado: {datetime.now().strftime('%d de %B de %Y a las %H:%M hrs')}</div>
        </div>
        
        <div class="content">
            <div class="summary-section">
                <h2 class="section-title">Resumen Ejecutivo</h2>
                <div class="summary-grid">
                    <div class="summary-card">
                        <div class="summary-value">{len(all_data)}</div>
                        <div class="summary-label">Documentos Procesados</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_pages}</div>
                        <div class="summary-label">Páginas Totales</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_chars:,}</div>
                        <div class="summary-label">Caracteres Extraídos</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_mop}</div>
                        <div class="summary-label">Códigos MOP Identificados</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_items}</div>
                        <div class="summary-label">Items Presupuestarios</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_chars//1000:.0f}K</div>
                        <div class="summary-label">Promedio Caracteres</div>
                    </div>
                </div>
            </div>
            
            <div class="documents-section">
                <h2 class="section-title">Detalle por Documento</h2>
                <table class="documents-table">
                    <thead>
                        <tr>
                            <th>📄 Documento</th>
                            <th>🏗️ Proyecto</th>
                            <th>📍 Ubicación</th>
                            <th>🔧 Tipo de Obra</th>
                            <th>🔢 Códigos MOP</th>
                            <th>📋 Items</th>
                            <th>📝 Contenido</th>
                        </tr>
                    </thead>
                    <tbody>
    """
    
    # Agregar filas de documentos
    for data in sorted(all_data, key=lambda x: x['filename']):
        s = data['summary']
        
        # Procesar datos
        comunas = ', '.join(s.get('comunas', [])[:2]) if s.get('comunas') else 'N/D'
        if len(s.get('comunas', [])) > 2:
            comunas += f" (+{len(s.get('comunas', [])) - 2} más)"
        
        proyecto = s.get('proyecto', 'No identificado')
        if proyecto == 'No identificado':
            proyecto = '<span class="no-data">No identificado</span>'
        else:
            proyecto = proyecto[:80] + ('...' if len(proyecto) > 80 else '')
        
        region = s.get('region', 'N/D')
        tipo_obra = s.get('tipo_obra', 'N/D')
        if tipo_obra != 'N/D':
            tipo_obra = tipo_obra[:40] + ('...' if len(tipo_obra) > 40 else '')
        
        mop_count = s.get('codigos_mop', 0)
        items_count = s.get('items_principales', 0)
        chars_count = data['extraction']['total_characters']
        
        html_content += f"""
                        <tr>
                            <td><div class="doc-name">{data['filename']}</div></td>
                            <td><div class="project-name">{proyecto}</div></td>
                            <td><div class="location">{region}<br>{comunas}</div></td>
                            <td>{tipo_obra}</td>
                            <td><span class="mop-count">{mop_count}</span></td>
                            <td><span class="items-count">{items_count}</span></td>
                            <td><div class="chars-count">{chars_count:,} chars</div></td>
                        </tr>
        """
    
    html_content += """
                    </tbody>
                </table>
            </div>
        </div>
        
        <div class="footer">
            <p>🤖 Generado automáticamente por MOP Analyzer v2.0</p>
            <p>Sistema de análisis de documentos de licitación del Ministerio de Obras Públicas</p>
        </div>
    </div>
</body>
</html>
    """
    
    # Guardar archivo
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    report_file = RESULTS_DIR / f"informe_consolidado_{timestamp}.html"
    
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    # También crear una versión sin timestamp
    report_file_simple = RESULTS_DIR / "informe_consolidado.html"
    with open(report_file_simple, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"\n✅ INFORME HTML GENERADO EXITOSAMENTE")
    print(f"📁 Archivo principal: {report_file}")
    print(f"📁 Acceso directo: {report_file_simple}")
    print(f"🌐 Para ver el informe, abre el archivo HTML en tu navegador")
    
    # Mostrar estadísticas finales
    print(f"\n📊 ESTADÍSTICAS DEL INFORME:")
    print(f"   📄 Documentos procesados: {len(all_data)}")
    print(f"   📝 Total caracteres: {total_chars:,}")
    print(f"   🔢 Total códigos MOP: {total_mop}")
    print(f"   📋 Total items: {total_items}")
    
    # Si estamos en Jupyter, mostrar el HTML
    try:
        from IPython.display import HTML, display
        display(HTML(f'<p style="color: green; font-weight: bold;">✅ Informe generado: <a href="{report_file.name}" target="_blank">{report_file.name}</a></p>'))
    except ImportError:
        pass
    
    return {
        'report_file': report_file,
        'report_file_simple': report_file_simple,
        'total_documents': len(all_data),
        'total_characters': total_chars,
        'total_mop_codes': total_mop,
        'total_items': total_items
    }

print("✅ Función de generación de informe HTML COMPLETA cargada")
print("\n🚀 USAR: generate_consolidated_report()")
print("📄 Generará un informe HTML profesional con todos los análisis")

In [ ]:
# Celda 12-b
generate_consolidated_report()

In [ ]:
# ============================================================================
# CELDA 13: DIAGNÓSTICO DE PROBLEMAS CON BASES3.PDF
# ============================================================================

def diagnose_pdf_issues(filename: str):
    """
    Diagnostica problemas con un PDF específico.
    """
    pdf_path = BASES_DIR / filename
    
    print(f"\n🔍 DIAGNÓSTICO: {filename}")
    print("="*80)
    
    if not pdf_path.exists():
        print(f"❌ Archivo no encontrado")
        return
    
    # Información básica
    size_mb = pdf_path.stat().st_size / 1024 / 1024
    print(f"📊 Tamaño: {size_mb:.2f} MB")
    
    # Intentar leer con PyPDF2
    try:
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            num_pages = len(reader.pages)
            print(f"📄 Páginas: {num_pages}")
            
            # Intentar extraer texto de las primeras páginas
            text_sample = ""
            for i in range(min(3, num_pages)):
                try:
                    page_text = reader.pages[i].extract_text()
                    text_sample += page_text
                    print(f"   Página {i+1}: {len(page_text)} caracteres")
                except Exception as e:
                    print(f"   Página {i+1}: Error - {e}")
            
            if text_sample:
                print(f"\n📝 Muestra de texto (primeros 500 caracteres):")
                print(text_sample[:500])
            else:
                print("\n⚠️ No se pudo extraer texto con PyPDF2")
                print("   El PDF podría ser:")
                print("   - Un documento escaneado (requiere OCR)")
                print("   - Un PDF protegido")
                print("   - Un PDF corrupto")
                
    except Exception as e:
        print(f"❌ Error leyendo PDF: {e}")
    
    # Verificar archivos existentes
    text_file = RESULTS_DIR / f"{pdf_path.stem}_texto.txt"
    if text_file.exists():
        with open(text_file, 'r', encoding='utf-8') as f:
            existing_text = f.read()
        print(f"\n📄 Archivo de texto existe: {len(existing_text)} caracteres")
        if len(existing_text) < 100:
            print("   ⚠️ El archivo de texto está casi vacío")
            print("   Recomendación: Eliminar y volver a procesar con OCR")
    
    return True

# Diagnosticar bases3.pdf que parece tener problemas
diagnose_pdf_issues("bases3.pdf")

In [ ]:
# ============================================================================
# CELDA 14: LIMPIEZA Y REPROCESAMIENTO FORZADO
# ============================================================================

def force_reprocess_problematic_files():
    """
    Reprocesa archivos que tienen 0 caracteres o análisis vacíos.
    """
    print("\n🔧 REPROCESAMIENTO FORZADO DE ARCHIVOS PROBLEMÁTICOS")
    print("="*80)
    
    problematic = []
    
    # Identificar archivos problemáticos
    for pdf_name in ["bases1.pdf", "bases2.pdf", "bases3.pdf"]:
        pdf_path = BASES_DIR / pdf_name
        text_file = RESULTS_DIR / f"{pdf_path.stem}_texto.txt"
        
        if text_file.exists():
            with open(text_file, 'r', encoding='utf-8') as f:
                text = f.read()
            if len(text) < 100:  # Menos de 100 caracteres es problemático
                problematic.append(pdf_name)
                print(f"   ⚠️ {pdf_name}: Solo {len(text)} caracteres")
    
    if not problematic:
        print("   ✅ No se encontraron archivos problemáticos")
        return
    
    print(f"\n   Archivos a reprocesar: {', '.join(problematic)}")
    
    for pdf_name in problematic:
        print(f"\n🔄 Reprocesando {pdf_name}...")
        
        # Eliminar archivos antiguos
        pdf_path = BASES_DIR / pdf_name
        text_file = RESULTS_DIR / f"{pdf_path.stem}_texto.txt"
        analysis_file = RESULTS_DIR / f"{pdf_path.stem}_analisis_completo.json"
        
        if text_file.exists():
            text_file.unlink()
            print(f"   🗑️ Eliminado texto anterior")
        
        if analysis_file.exists():
            analysis_file.unlink()
            print(f"   🗑️ Eliminado análisis anterior")
        
        # Reprocesar
        result = analyze_pdf_document(pdf_path, force_ocr=True)
        
        if result['success'] and result['extraction']['total_characters'] > 100:
            print(f"   ✅ Reprocesamiento exitoso: {result['extraction']['total_characters']:,} caracteres")
        else:
            print(f"   ❌ Falló el reprocesamiento")

print("✅ Función de reprocesamiento forzado cargada")
print("\nUSO: force_reprocess_problematic_files()")

In [ ]:
# ============================================================================
# CELDA 15: EJECUTOR MAESTRO DEL SISTEMA
# ============================================================================

def run_complete_system_analysis():
    """
    Ejecuta el análisis completo del sistema con todas las optimizaciones.
    """
    print("\n" + "="*80)
    print("🚀 SISTEMA MOP ANALYZER v2.0 - ANÁLISIS COMPLETO")
    print("="*80)
    
    start_time = time.time()
    
    # PASO 1: Verificación inicial
    print("\n📁 PASO 1: Verificación del sistema")
    print("-" * 40)
    check_existing_files()
    
    # PASO 2: Procesar todos los documentos
    print("\n📄 PASO 2: Procesamiento paralelo de documentos")
    print("-" * 40)
    results = process_all_bases_complete()
    
    # PASO 3: Generar informe consolidado
    print("\n📊 PASO 3: Generación de informes")
    print("-" * 40)
    
    if results:
        report = generate_consolidated_report()
        
        # Estadísticas finales
        elapsed = time.time() - start_time
        print(f"\n{'='*80}")
        print("✅ ANÁLISIS COMPLETO FINALIZADO")
        print(f"{'='*80}")
        print(f"   ⏱️ Tiempo total: {elapsed:.1f} segundos ({elapsed/60:.1f} minutos)")
        print(f"   📄 Documentos procesados: {len(results)}")
        print(f"   📊 Informe disponible en: {report}")
        print(f"   🚀 Velocidad promedio: {elapsed/len(results):.1f}s por documento")
    else:
        print("❌ No se procesaron documentos")
    
    return results

print("\n" + "="*80)
print("✅ SISTEMA MOP ANALYZER v2.0 - CARGADO COMPLETAMENTE")
print("="*80)
print("\n🎯 COMANDOS PRINCIPALES:")
print("\n1. ANÁLISIS COMPLETO DE TODOS LOS PDFs:")
print("   >>> run_complete_system_analysis()")
print("\n2. Procesar todos los archivos:")
print("   >>> process_all_bases_complete()")
print("\n3. Reprocesar archivo con más workers:")
print("   >>> reprocess_file_enhanced('bases3.pdf', max_workers=6, chunk_size=10)")
print("\n4. Verificar estado:")
print("   >>> check_existing_files()")
print("\n" + "="*80)

In [ ]:
# Ejecutar el análisis completo del sistema
run_complete_system_analysis()

In [ ]:
# ============================================================================
# GENERAR INFORME HTML DESDE DATOS EXISTENTES
# ============================================================================

def generate_html_from_results(results_data):
    """
    Genera un informe HTML desde los datos de resultados ya procesados.
    """
    print("\n📊 GENERANDO INFORME HTML DESDE DATOS EXISTENTES")
    print("="*80)
    
    if not results_data:
        print("❌ No hay datos para generar el informe")
        return None
    
    # Estadísticas globales
    total_docs = len(results_data)
    total_chars = sum(data['extraction']['total_characters'] for data in results_data)
    total_mop = sum(data['summary'].get('codigos_mop', 0) for data in results_data)
    total_items = sum(data['summary'].get('items_principales', 0) for data in results_data)
    total_ete = sum(data['summary'].get('codigos_ete', 0) for data in results_data)
    total_safi = sum(data['summary'].get('codigos_safi', 0) for data in results_data)
    
    # Crear HTML completo
    html_content = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>📊 Informe Consolidado MOP - Análisis de Presupuestos</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: 'Segoe UI', -apple-system, BlinkMacSystemFont, Arial, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
            line-height: 1.6;
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }}
        
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 40px;
            text-align: center;
        }}
        
        .header h1 {{
            font-size: 2.8em;
            margin-bottom: 15px;
            font-weight: 300;
            text-shadow: 0 2px 4px rgba(0,0,0,0.3);
        }}
        
        .header .subtitle {{
            font-size: 1.3em;
            opacity: 0.9;
            margin-bottom: 20px;
        }}
        
        .header .date {{
            font-size: 1em;
            opacity: 0.8;
            background: rgba(255,255,255,0.1);
            padding: 10px 20px;
            border-radius: 25px;
            display: inline-block;
        }}
        
        .content {{
            padding: 40px;
        }}
        
        .summary-section {{
            margin-bottom: 50px;
        }}
        
        .section-title {{
            font-size: 2em;
            color: #333;
            margin-bottom: 30px;
            text-align: center;
            position: relative;
        }}
        
        .section-title::after {{
            content: '';
            position: absolute;
            bottom: -10px;
            left: 50%;
            transform: translateX(-50%);
            width: 60px;
            height: 3px;
            background: linear-gradient(135deg, #667eea, #764ba2);
            border-radius: 2px;
        }}
        
        .summary-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 25px;
            margin: 40px 0;
        }}
        
        .summary-card {{
            background: white;
            border-radius: 15px;
            padding: 25px;
            text-align: center;
            box-shadow: 0 8px 25px rgba(0,0,0,0.1);
            transition: all 0.3s ease;
            border: 2px solid transparent;
            position: relative;
            overflow: hidden;
        }}
        
        .summary-card::before {{
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            height: 4px;
            background: linear-gradient(135deg, #667eea, #764ba2);
        }}
        
        .summary-card:hover {{
            transform: translateY(-5px);
            box-shadow: 0 15px 35px rgba(0,0,0,0.15);
        }}
        
        .summary-value {{
            font-size: 2.2em;
            font-weight: 700;
            background: linear-gradient(135deg, #667eea, #764ba2);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
            margin-bottom: 8px;
        }}
        
        .summary-label {{
            color: #666;
            font-size: 0.95em;
            font-weight: 500;
            text-transform: uppercase;
            letter-spacing: 1px;
        }}
        
        .documents-table {{
            width: 100%;
            background: white;
            border-radius: 15px;
            overflow: hidden;
            box-shadow: 0 8px 25px rgba(0,0,0,0.1);
            margin-top: 30px;
        }}
        
        .documents-table th {{
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
            padding: 18px 12px;
            text-align: left;
            font-weight: 600;
            text-transform: uppercase;
            letter-spacing: 0.5px;
            font-size: 0.85em;
        }}
        
        .documents-table td {{
            padding: 15px 12px;
            border-bottom: 1px solid #eee;
            vertical-align: top;
            font-size: 0.9em;
        }}
        
        .documents-table tr:last-child td {{
            border-bottom: none;
        }}
        
        .documents-table tbody tr:hover {{
            background: #f8f9ff;
        }}
        
        .doc-name {{
            font-weight: 600;
            color: #333;
            font-size: 1em;
        }}
        
        .project-name {{
            color: #555;
            max-width: 250px;
            word-wrap: break-word;
            line-height: 1.4;
        }}
        
        .no-data {{
            color: #999;
            font-style: italic;
        }}
        
        .badge {{
            padding: 4px 10px;
            border-radius: 15px;
            font-size: 0.8em;
            font-weight: 600;
            display: inline-block;
            margin: 2px;
        }}
        
        .badge-mop {{
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
        }}
        
        .badge-ete {{
            background: #28a745;
            color: white;
        }}
        
        .badge-safi {{
            background: #ffc107;
            color: #333;
        }}
        
        .chars-count {{
            color: #666;
            font-size: 0.85em;
        }}
        
        .location {{
            color: #555;
            font-size: 0.9em;
            line-height: 1.3;
        }}
        
        .presupuesto {{
            color: #28a745;
            font-weight: 600;
        }}
        
        .footer {{
            text-align: center;
            padding: 30px;
            background: #f8f9fa;
            color: #666;
            font-size: 0.9em;
        }}
        
        .codes-section {{
            margin-top: 40px;
        }}
        
        .codes-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
            gap: 25px;
            margin: 30px 0;
        }}
        
        .codes-card {{
            background: white;
            border-radius: 12px;
            padding: 20px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
        }}
        
        .codes-title {{
            font-weight: 600;
            color: #333;
            margin-bottom: 15px;
            font-size: 1.1em;
        }}
        
        .code-item {{
            background: #f8f9fa;
            padding: 8px 12px;
            margin: 5px 0;
            border-radius: 8px;
            font-family: 'Courier New', monospace;
            font-size: 0.9em;
            border-left: 3px solid #667eea;
        }}
        
        @media (max-width: 768px) {{
            .header h1 {{ font-size: 2em; }}
            .summary-grid {{ grid-template-columns: repeat(2, 1fr); }}
            .content {{ padding: 20px; }}
            .documents-table {{ font-size: 0.8em; }}
            .documents-table th, .documents-table td {{ padding: 10px 6px; }}
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>📊 Informe Consolidado MOP</h1>
            <div class="subtitle">Análisis de Bases de Licitación y Presupuestos</div>
            <div class="date">📅 Generado: {datetime.now().strftime('%d de %B de %Y a las %H:%M hrs')}</div>
        </div>
        
        <div class="content">
            <div class="summary-section">
                <h2 class="section-title">📈 Resumen Ejecutivo</h2>
                <div class="summary-grid">
                    <div class="summary-card">
                        <div class="summary-value">{total_docs}</div>
                        <div class="summary-label">📄 Documentos</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_chars:,}</div>
                        <div class="summary-label">📝 Caracteres</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_mop}</div>
                        <div class="summary-label">🔢 Códigos MOP</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_ete}</div>
                        <div class="summary-label">📋 Códigos ETE</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_safi}</div>
                        <div class="summary-label">💼 Códigos SAFI</div>
                    </div>
                    <div class="summary-card">
                        <div class="summary-value">{total_items}</div>
                        <div class="summary-label">📊 Items Presup.</div>
                    </div>
                </div>
            </div>
            
            <div class="documents-section">
                <h2 class="section-title">📋 Detalle por Documento</h2>
                <table class="documents-table">
                    <thead>
                        <tr>
                            <th>📄 Archivo</th>
                            <th>🏗️ Proyecto</th>
                            <th>📍 Ubicación</th>
                            <th>🔧 Tipo de Obra</th>
                            <th>💰 Presupuesto</th>
                            <th>🔢 Códigos</th>
                            <th>📝 Contenido</th>
                        </tr>
                    </thead>
                    <tbody>
    """
    
    # Agregar filas de documentos
    for data in sorted(results_data, key=lambda x: x['filename']):
        s = data['summary']
        
        # Procesar datos
        comunas = ', '.join(s.get('comunas', [])[:2]) if s.get('comunas') else 'N/D'
        if len(s.get('comunas', [])) > 2:
            comunas += f" (+{len(s.get('comunas', [])) - 2})"
        
        proyecto = s.get('proyecto', 'No identificado')
        if proyecto == 'No identificado':
            proyecto = '<span class="no-data">No identificado</span>'
        else:
            proyecto = proyecto[:70] + ('...' if len(proyecto) > 70 else '')
        
        region = s.get('region', 'N/D')
        tipo_obra = s.get('tipo_obra', 'N/D')
        if tipo_obra != 'N/D' and len(tipo_obra) > 35:
            tipo_obra = tipo_obra[:35] + '...'
        
        presupuesto = s.get('presupuesto_estimado', 'N/D')
        if presupuesto == 'N/D':
            presupuesto = '<span class="no-data">N/D</span>'
        else:
            presupuesto = f'<span class="presupuesto">{presupuesto}</span>'
        
        mop_count = s.get('codigos_mop', 0)
        ete_count = s.get('codigos_ete', 0)
        safi_count = s.get('codigos_safi', 0)
        chars_count = data['extraction']['total_characters']
        
        # Construir badges de códigos
        codes_html = ""
        if mop_count > 0:
            codes_html += f'<span class="badge badge-mop">MOP: {mop_count}</span>'
        if ete_count > 0:
            codes_html += f'<span class="badge badge-ete">ETE: {ete_count}</span>'
        if safi_count > 0:
            codes_html += f'<span class="badge badge-safi">SAFI: {safi_count}</span>'
        
        if not codes_html:
            codes_html = '<span class="no-data">Sin códigos</span>'
        
        html_content += f"""
                        <tr>
                            <td><div class="doc-name">{data['filename']}</div></td>
                            <td><div class="project-name">{proyecto}</div></td>
                            <td><div class="location">{region}<br><small>{comunas}</small></div></td>
                            <td>{tipo_obra}</td>
                            <td>{presupuesto}</td>
                            <td>{codes_html}</td>
                            <td><div class="chars-count">{chars_count:,} chars<br>{data['extraction'].get('total_words', 0):,} palabras</div></td>
                        </tr>
        """
    
    # Agregar sección de códigos detallados
    html_content += """
                    </tbody>
                </table>
            </div>
            
            <div class="codes-section">
                <h2 class="section-title">🔍 Códigos Identificados por Documento</h2>
                <div class="codes-grid">
    """
    
    for data in results_data:
        patterns = data.get('patterns_extracted', {})
        filename = data['filename']
        
        html_content += f"""
                    <div class="codes-card">
                        <div class="codes-title">📄 {filename}</div>
        """
        
        if patterns.get('mop_codes'):
            html_content += f"""
                        <div style="margin-bottom: 15px;">
                            <strong>🔢 Códigos MOP ({len(patterns['mop_codes'])}):</strong>
            """
            for code in patterns['mop_codes'][:10]:  # Mostrar máximo 10
                html_content += f'<div class="code-item">{code}</div>'
            if len(patterns['mop_codes']) > 10:
                html_content += f'<div class="no-data">... y {len(patterns["mop_codes"]) - 10} más</div>'
            html_content += '</div>'
        
        if patterns.get('ete_codes'):
            html_content += f"""
                        <div style="margin-bottom: 15px;">
                            <strong>📋 Códigos ETE ({len(patterns['ete_codes'])}):</strong>
            """
            for code in patterns['ete_codes']:
                html_content += f'<div class="code-item">{code}</div>'
            html_content += '</div>'
        
        if patterns.get('safi_codes'):
            html_content += f"""
                        <div style="margin-bottom: 15px;">
                            <strong>💼 Códigos SAFI ({len(patterns['safi_codes'])}):</strong>
            """
            for code in patterns['safi_codes']:
                html_content += f'<div class="code-item">{code}</div>'
            html_content += '</div>'
        
        html_content += '</div>'
    
    html_content += """
                </div>
            </div>
        </div>
        
        <div class="footer">
            <p>🤖 Generado automáticamente por MOP Analyzer v2.0</p>
            <p>Sistema de análisis de documentos de licitación del Ministerio de Obras Públicas</p>
        </div>
    </div>
</body>
</html>
    """
    
    # Guardar archivo
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    report_file = RESULTS_DIR / f"informe_consolidado_{timestamp}.html"
    
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    # También crear una versión sin timestamp
    report_file_simple = RESULTS_DIR / "informe_consolidado.html"
    with open(report_file_simple, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"\n✅ INFORME HTML GENERADO EXITOSAMENTE")
    print(f"📁 Archivo con timestamp: {report_file}")
    print(f"📁 Archivo principal: {report_file_simple}")
    print(f"🌐 Para ver el informe, abre cualquiera de los archivos HTML en tu navegador")
    
    # Mostrar estadísticas finales
    print(f"\n📊 ESTADÍSTICAS DEL INFORME:")
    print(f"   📄 Documentos procesados: {total_docs}")
    print(f"   📝 Total caracteres: {total_chars:,}")
    print(f"   🔢 Total códigos MOP: {total_mop}")
    print(f"   📋 Total códigos ETE: {total_ete}")
    print(f"   💼 Total códigos SAFI: {total_safi}")
    print(f"   📊 Total items presupuestarios: {total_items}")
    
    return {
        'report_file': report_file,
        'report_file_simple': report_file_simple,
        'total_documents': total_docs,
        'total_characters': total_chars,
        'total_mop_codes': total_mop,
        'total_ete_codes': total_ete,
        'total_safi_codes': total_safi,
        'total_items': total_items
    }

print("✅ Función para generar HTML desde datos existentes cargada")
print("\n🚀 EJECUTAR:")
print("generate_html_from_results(results)")

# Ejecutar automáticamente si tienes la variable 'results' disponible
try:
    # Intentar usar la variable 'results' del output anterior
    if 'results' in locals() or 'results' in globals():
        print("\n🔄 Detectados resultados existentes, generando informe...")
        generate_html_from_results(results)
    else:
        print("\n⚠️ Para generar el informe, ejecuta:")
        print("generate_html_from_results(tus_datos_de_resultados)")
except Exception as e:
    print(f"\n⚠️ Para generar el informe, ejecuta:")
    print("generate_html_from_results(tus_datos_de_resultados)")
    print(f"Error detectado: {e}")

In [ ]:
generate_html_from_results(results)


In [5]:
# ============================================================================
# VERSIÓN OPTIMIZADA - PROCESADOR OCR PARA PDFS MOP
# ============================================================================

import os
import json
import re
import time
import requests
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
import PyPDF2
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Markdown
import anthropic
from dotenv import load_dotenv
import hashlib
import pickle
from functools import lru_cache
import asyncio
import aiohttp
from tqdm.notebook import tqdm

# ============================================================================
# CONFIGURACIÓN OPTIMIZADA
# ============================================================================

# Cargar variables de entorno
load_dotenv(Path('.env') if Path('.env').exists() else Path('../.env'))

PDF_REST_API_KEY = os.getenv('PDF_REST_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

if not PDF_REST_API_KEY:
    raise ValueError("❌ PDF_REST_API_KEY no encontrada en .env")

# Configuración de rutas
BASE_DIR = Path("storage/projects/conservacion_caminos")
BASES_DIR = BASE_DIR / "bases"
RESULTS_DIR = BASE_DIR / "results"
TEMP_DIR = BASE_DIR / "temp"
CACHE_DIR = BASE_DIR / "cache"  # Nuevo directorio de caché

# Crear directorios
for dir_path in [RESULTS_DIR, TEMP_DIR, CACHE_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Cliente Anthropic (opcional - solo si necesitas análisis IA)
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY) if ANTHROPIC_API_KEY else None

# ============================================================================
# CONFIGURACIÓN OPTIMIZADA DEL SISTEMA
# ============================================================================

CONFIG = {
    'CHUNK_SIZE': 10,        # Reducido de 30 a 10 páginas
    'MAX_WORKERS': 8,        # Aumentado de 3-4 a 8 workers
    'OCR_TIMEOUT': 120,      # Reducido de 300-600 a 120 segundos
    'RETRY_COUNT': 1,        # Reducido de 2 a 1 reintento
    'USE_CACHE': True,       # Activar caché
    'PARALLEL_MODE': 'thread',  # 'thread' o 'process'
    'BATCH_SIZE': 5,         # Procesar en batches
    'MIN_VALID_TEXT': 500,   # Mínimo de caracteres
}

print("✅ Configuración optimizada cargada")
print(f"   📁 Cache: {CACHE_DIR}")
print(f"   ⚡ Workers: {CONFIG['MAX_WORKERS']}")
print(f"   📄 Chunk size: {CONFIG['CHUNK_SIZE']} páginas")

# ============================================================================
# SISTEMA DE CACHÉ INTELIGENTE
# ============================================================================

class SmartCache:
    """Sistema de caché para evitar reprocesar chunks."""
    
    def __init__(self, cache_dir: Path):
        self.cache_dir = cache_dir
        self.index_file = cache_dir / "cache_index.json"
        self.index = self._load_index()
    
    def _load_index(self):
        if self.index_file.exists():
            with open(self.index_file, 'r') as f:
                return json.load(f)
        return {}
    
    def _save_index(self):
        with open(self.index_file, 'w') as f:
            json.dump(self.index, f)
    
    def get_hash(self, file_path: Path, start_page: int, end_page: int) -> str:
        """Genera hash único para un chunk."""
        key = f"{file_path.name}_{start_page}_{end_page}_{file_path.stat().st_mtime}"
        return hashlib.md5(key.encode()).hexdigest()
    
    def get(self, hash_key: str) -> Optional[Dict]:
        """Recupera resultado cacheado si existe."""
        if hash_key in self.index:
            cache_file = self.cache_dir / f"{hash_key}.pkl"
            if cache_file.exists():
                with open(cache_file, 'rb') as f:
                    return pickle.load(f)
        return None
    
    def set(self, hash_key: str, data: Dict):
        """Guarda resultado en caché."""
        cache_file = self.cache_dir / f"{hash_key}.pkl"
        with open(cache_file, 'wb') as f:
            pickle.dump(data, f)
        self.index[hash_key] = time.time()
        self._save_index()

cache = SmartCache(CACHE_DIR)

# ============================================================================
# SPLITTER OPTIMIZADO
# ============================================================================

class OptimizedPDFSplitter:
    """División optimizada de PDFs."""
    
    def __init__(self, pages_per_chunk: int = 10):
        self.pages_per_chunk = pages_per_chunk
    
    def split_pdf_fast(self, pdf_path: Path) -> List[Tuple[Path, int, int]]:
        """División rápida de PDF sin escribir chunks intermedios si no es necesario."""
        chunks_info = []
        
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            total_pages = len(pdf_reader.pages)
        
        print(f"📄 PDF: {pdf_path.name} ({total_pages} páginas)")
        
        # Crear directorio para chunks
        chunks_dir = TEMP_DIR / f"{pdf_path.stem}_chunks"
        chunks_dir.mkdir(exist_ok=True)
        
        # Dividir en chunks más pequeños
        for i, start in enumerate(range(0, total_pages, self.pages_per_chunk), 1):
            end = min(start + self.pages_per_chunk, total_pages)
            chunk_path = chunks_dir / f"{pdf_path.stem}_chunk_{i:03d}.pdf"
            
            # Solo crear el chunk si no está cacheado
            cache_key = cache.get_hash(pdf_path, start, end)
            if CONFIG['USE_CACHE'] and cache.get(cache_key):
                print(f"   📦 Chunk {i}: páginas {start+1}-{end} [CACHEADO]")
            else:
                # Crear chunk
                with open(pdf_path, 'rb') as input_file:
                    reader = PyPDF2.PdfReader(input_file)
                    writer = PyPDF2.PdfWriter()
                    
                    for page_num in range(start, end):
                        writer.add_page(reader.pages[page_num])
                    
                    with open(chunk_path, 'wb') as output_file:
                        writer.write(output_file)
                
                print(f"   📦 Chunk {i}: páginas {start+1}-{end}")
            
            chunks_info.append((chunk_path, start + 1, end, cache_key))
        
        return chunks_info

# ============================================================================
# OCR OPTIMIZADO CON BATCHING
# ============================================================================

def apply_ocr_optimized(chunk_info: Tuple) -> Dict[str, Any]:
    """OCR optimizado con caché y timeouts reducidos."""
    chunk_path, start_page, end_page, cache_key = chunk_info
    
    # Verificar caché
    if CONFIG['USE_CACHE']:
        cached = cache.get(cache_key)
        if cached:
            return cached
    
    result = {
        "chunk_name": chunk_path.name,
        "pages": (start_page, end_page),
        "success": False,
        "text": "",
        "error": None,
        "processing_time": 0,
        "characters": 0
    }
    
    start_time = time.time()
    
    # Si el archivo no existe (porque estaba cacheado), retornar caché vacío
    if not chunk_path.exists():
        result["error"] = "Chunk file not found (likely cached)"
        return result
    
    try:
        # OCR con timeout reducido
        ocr_url = "https://api.pdfrest.com/pdf-with-ocr-text"
        
        with open(chunk_path, 'rb') as file:
            files = [('file', (chunk_path.name, file, 'application/pdf'))]
            headers = {'Api-Key': PDF_REST_API_KEY}
            payload = {
                'output': f'ocr_{chunk_path.stem}',
                'languages': 'Spanish'
            }
            
            response = requests.post(
                ocr_url,
                headers=headers,
                data=payload,
                files=files,
                timeout=CONFIG['OCR_TIMEOUT']
            )
        
        if response.status_code == 200:
            data = response.json()
            output_url = data.get('outputUrl')
            
            if output_url:
                # Descargar y extraer texto
                pdf_response = requests.get(output_url, timeout=30)
                
                if pdf_response.status_code == 200:
                    # Guardar temporalmente
                    temp_pdf = TEMP_DIR / f"temp_{chunk_path.stem}.pdf"
                    with open(temp_pdf, 'wb') as f:
                        f.write(pdf_response.content)
                    
                    # Extraer texto
                    extract_url = "https://api.pdfrest.com/extracted-text"
                    with open(temp_pdf, 'rb') as file:
                        files = [('file', (temp_pdf.name, file, 'application/pdf'))]
                        response = requests.post(
                            extract_url, 
                            headers=headers, 
                            files=files, 
                            timeout=30
                        )
                    
                    if response.status_code == 200:
                        text = response.json().get('fullText', '')
                        text = re.sub(r'\[pdfRest.*?\]', '', text)
                        
                        result["success"] = True
                        result["text"] = text
                        result["characters"] = len(text)
                    
                    # Limpiar temporal
                    if temp_pdf.exists():
                        temp_pdf.unlink()
    
    except requests.Timeout:
        result["error"] = f"Timeout ({CONFIG['OCR_TIMEOUT']}s)"
    except Exception as e:
        result["error"] = str(e)[:100]
    
    result["processing_time"] = time.time() - start_time
    
    # Guardar en caché si fue exitoso
    if CONFIG['USE_CACHE'] and result["success"]:
        cache.set(cache_key, result)
    
    return result

# ============================================================================
# PROCESAMIENTO PARALELO OPTIMIZADO
# ============================================================================

def process_chunks_batch_parallel(chunks_info: List[Tuple], max_workers: int = None) -> Dict:
    """Procesamiento paralelo optimizado con batching."""
    if max_workers is None:
        max_workers = CONFIG['MAX_WORKERS']
    
    print(f"\n⚡ Procesando {len(chunks_info)} chunks con {max_workers} workers")
    
    results = []
    texts_by_page = {}
    successful = 0
    failed = 0
    cached = 0
    
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Procesar en batches para mejor control
        batch_size = CONFIG['BATCH_SIZE']
        
        with tqdm(total=len(chunks_info), desc="Procesando chunks") as pbar:
            for i in range(0, len(chunks_info), batch_size):
                batch = chunks_info[i:i+batch_size]
                
                # Enviar batch
                futures = {
                    executor.submit(apply_ocr_optimized, chunk): chunk 
                    for chunk in batch
                }
                
                # Procesar resultados del batch
                for future in as_completed(futures):
                    chunk_info = futures[future]
                    try:
                        result = future.result(timeout=CONFIG['OCR_TIMEOUT'] + 10)
                        results.append(result)
                        
                        if result["success"]:
                            start_page = result["pages"][0]
                            texts_by_page[start_page] = result["text"]
                            successful += 1
                            
                            # Verificar si vino de caché
                            if result["processing_time"] < 0.1:
                                cached += 1
                        else:
                            failed += 1
                    
                    except Exception as e:
                        failed += 1
                        print(f"❌ Error: {str(e)[:50]}")
                    
                    pbar.update(1)
    
    # Consolidar texto en orden
    consolidated_text = ""
    for page_num in sorted(texts_by_page.keys()):
        consolidated_text += f"\n\n--- Páginas {page_num} ---\n"
        consolidated_text += texts_by_page[page_num]
    
    elapsed = time.time() - start_time
    
    print(f"\n📊 Resultados:")
    print(f"   ✅ Exitosos: {successful}/{len(chunks_info)}")
    print(f"   💾 Desde caché: {cached}")
    print(f"   ❌ Fallidos: {failed}")
    print(f"   ⏱️ Tiempo: {elapsed:.1f}s ({elapsed/len(chunks_info):.1f}s/chunk)")
    
    return {
        "success": successful > 0,
        "text": consolidated_text,
        "chunks_processed": len(chunks_info),
        "chunks_successful": successful,
        "chunks_failed": failed,
        "chunks_cached": cached,
        "total_characters": len(consolidated_text),
        "processing_time": elapsed
    }

# ============================================================================
# FUNCIÓN PRINCIPAL OPTIMIZADA
# ============================================================================

def process_pdf_fast(pdf_path: Path, skip_ai: bool = True) -> Dict:
    """Procesa un PDF de forma optimizada."""
    print(f"\n{'='*70}")
    print(f"⚡ PROCESAMIENTO RÁPIDO: {pdf_path.name}")
    print(f"{'='*70}")
    
    total_start = time.time()
    
    # Verificar texto existente
    text_file = RESULTS_DIR / f"{pdf_path.stem}_texto.txt"
    if text_file.exists() and not CONFIG.get('FORCE_REPROCESS', False):
        print(f"✅ Texto ya existe: {text_file.name}")
        with open(text_file, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
        return {
            "success": True,
            "text": text,
            "cached": True,
            "processing_time": 0
        }
    
    # Dividir PDF
    splitter = OptimizedPDFSplitter(pages_per_chunk=CONFIG['CHUNK_SIZE'])
    chunks_info = splitter.split_pdf_fast(pdf_path)
    
    # Procesar chunks en paralelo
    result = process_chunks_batch_parallel(chunks_info)
    
    if result["success"]:
        # Guardar texto
        text = result["text"]
        with open(text_file, 'w', encoding='utf-8', errors='ignore') as f:
            f.write(text)
        print(f"💾 Guardado: {text_file.name}")
        
        # Extraer patrones básicos (rápido)
        patterns = extract_patterns_quick(text)
        
        # Guardar resumen básico
        summary = {
            "filename": pdf_path.name,
            "timestamp": datetime.now().isoformat(),
            "total_characters": len(text),
            "processing_time": time.time() - total_start,
            "chunks_cached": result.get("chunks_cached", 0),
            "patterns": {
                "mop_codes": len(patterns.get('mop_codes', [])),
                "ete_codes": len(patterns.get('ete_codes', [])),
                "montos": len(patterns.get('montos', []))
            }
        }
        
        summary_file = RESULTS_DIR / f"{pdf_path.stem}_resumen_rapido.json"
        with open(summary_file, 'w') as f:
            json.dump(summary, f, indent=2)
        
        print(f"\n✅ Completado en {time.time() - total_start:.1f}s")
        return {
            "success": True,
            "text": text,
            "summary": summary,
            "processing_time": time.time() - total_start
        }
    
    return {"success": False, "error": "Falló el procesamiento"}

def extract_patterns_quick(text: str) -> Dict:
    """Extracción rápida de patrones sin regex complejos."""
    return {
        'mop_codes': re.findall(r'7\.\d{3}\.\d+', text)[:100],  # Limitar resultados
        'ete_codes': re.findall(r'ETE[\.\-\s]?\d+', text, re.IGNORECASE)[:50],
        'montos': re.findall(r'\$\s*[\d\.,]+', text)[:100]
    }

# ============================================================================
# PROCESAMIENTO EN BATCH OPTIMIZADO
# ============================================================================

def process_all_pdfs_fast():
    """Procesa todos los PDFs de forma optimizada."""
    print("\n" + "="*80)
    print("⚡ PROCESAMIENTO BATCH OPTIMIZADO")
    print("="*80)
    
    start_time = time.time()
    
    # Buscar PDFs
    pdf_files = list(BASES_DIR.glob("*.pdf"))
    
    if not pdf_files:
        print("❌ No se encontraron PDFs")
        return []
    
    print(f"📚 Archivos encontrados: {len(pdf_files)}")
    for pdf in pdf_files:
        size_mb = pdf.stat().st_size / 1024 / 1024
        print(f"   - {pdf.name} ({size_mb:.1f} MB)")
    
    # Procesar todos
    results = []
    for idx, pdf_path in enumerate(pdf_files, 1):
        print(f"\n[{idx}/{len(pdf_files)}] Procesando: {pdf_path.name}")
        result = process_pdf_fast(pdf_path, skip_ai=True)
        results.append(result)
        
        if result["success"]:
            print(f"   ✅ {result['summary']['total_characters']:,} caracteres")
            print(f"   ⏱️ {result['processing_time']:.1f}s")
    
    # Resumen final
    total_time = time.time() - start_time
    successful = sum(1 for r in results if r["success"])
    
    print(f"\n" + "="*80)
    print(f"📊 RESUMEN FINAL")
    print(f"="*80)
    print(f"✅ Exitosos: {successful}/{len(pdf_files)}")
    print(f"⏱️ Tiempo total: {total_time:.1f}s")
    print(f"⚡ Promedio: {total_time/len(pdf_files):.1f}s por archivo")
    
    # Generar tabla resumen
    summary_data = []
    for pdf, result in zip(pdf_files, results):
        if result["success"]:
            summary_data.append({
                'Archivo': pdf.name,
                'Tamaño MB': round(pdf.stat().st_size / 1024 / 1024, 1),
                'Caracteres': result['summary']['total_characters'],
                'Códigos MOP': result['summary']['patterns']['mop_codes'],
                'Tiempo (s)': round(result['processing_time'], 1),
                'Chunks Cache': result['summary'].get('chunks_cached', 0)
            })
    
    if summary_data:
        df = pd.DataFrame(summary_data)
        display(df)
        
        # Guardar Excel
        excel_file = RESULTS_DIR / f"resumen_optimizado_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
        df.to_excel(excel_file, index=False)
        print(f"\n💾 Resumen guardado: {excel_file}")
    
    return results

# ============================================================================
# FUNCIÓN DE LIMPIEZA
# ============================================================================

def cleanup_temp_files():
    """Limpia archivos temporales."""
    print("🧹 Limpiando archivos temporales...")
    
    # Limpiar chunks
    for chunk_dir in TEMP_DIR.glob("*_chunks"):
        for file in chunk_dir.glob("*.pdf"):
            file.unlink()
        chunk_dir.rmdir()
    
    # Limpiar PDFs temporales
    for temp_pdf in TEMP_DIR.glob("temp_*.pdf"):
        temp_pdf.unlink()
    
    print("✅ Limpieza completada")

print("\n" + "="*80)
print("✅ SISTEMA OPTIMIZADO LISTO")
print("="*80)
print("\nEjecuta:")
print("  >>> results = process_all_pdfs_fast()")
print("\nEsto debería completarse en menos de 10 minutos")
print("="*80)

✅ Configuración optimizada cargada
   📁 Cache: storage/projects/conservacion_caminos/cache
   ⚡ Workers: 8
   📄 Chunk size: 10 páginas

✅ SISTEMA OPTIMIZADO LISTO

Ejecuta:
  >>> results = process_all_pdfs_fast()

Esto debería completarse en menos de 10 minutos


In [6]:
results = process_all_pdfs_fast()



⚡ PROCESAMIENTO BATCH OPTIMIZADO
📚 Archivos encontrados: 3
   - bases2.pdf (7.6 MB)
   - bases3.pdf (17.3 MB)
   - bases1.pdf (12.2 MB)

[1/3] Procesando: bases2.pdf

⚡ PROCESAMIENTO RÁPIDO: bases2.pdf
📄 PDF: bases2.pdf (150 páginas)
   📦 Chunk 1: páginas 1-10
   📦 Chunk 2: páginas 11-20
   📦 Chunk 3: páginas 21-30
   📦 Chunk 4: páginas 31-40
   📦 Chunk 5: páginas 41-50
   📦 Chunk 6: páginas 51-60
   📦 Chunk 7: páginas 61-70
   📦 Chunk 8: páginas 71-80
   📦 Chunk 9: páginas 81-90
   📦 Chunk 10: páginas 91-100
   📦 Chunk 11: páginas 101-110
   📦 Chunk 12: páginas 111-120
   📦 Chunk 13: páginas 121-130
   📦 Chunk 14: páginas 131-140
   📦 Chunk 15: páginas 141-150

⚡ Procesando 15 chunks con 8 workers


Procesando chunks:   0%|          | 0/15 [00:00<?, ?it/s]


📊 Resultados:
   ✅ Exitosos: 15/15
   💾 Desde caché: 0
   ❌ Fallidos: 0
   ⏱️ Tiempo: 170.7s (11.4s/chunk)
💾 Guardado: bases2_texto.txt

✅ Completado en 171.2s
   ✅ 338,247 caracteres
   ⏱️ 171.2s

[2/3] Procesando: bases3.pdf

⚡ PROCESAMIENTO RÁPIDO: bases3.pdf
📄 PDF: bases3.pdf (135 páginas)
   📦 Chunk 1: páginas 1-10
   📦 Chunk 2: páginas 11-20
   📦 Chunk 3: páginas 21-30
   📦 Chunk 4: páginas 31-40
   📦 Chunk 5: páginas 41-50
   📦 Chunk 6: páginas 51-60
   📦 Chunk 7: páginas 61-70
   📦 Chunk 8: páginas 71-80
   📦 Chunk 9: páginas 81-90
   📦 Chunk 10: páginas 91-100
   📦 Chunk 11: páginas 101-110
   📦 Chunk 12: páginas 111-120
   📦 Chunk 13: páginas 121-130
   📦 Chunk 14: páginas 131-135

⚡ Procesando 14 chunks con 8 workers


Procesando chunks:   0%|          | 0/14 [00:00<?, ?it/s]


📊 Resultados:
   ✅ Exitosos: 14/14
   💾 Desde caché: 0
   ❌ Fallidos: 0
   ⏱️ Tiempo: 205.7s (14.7s/chunk)
💾 Guardado: bases3_texto.txt

✅ Completado en 206.2s
   ✅ 308,486 caracteres
   ⏱️ 206.2s

[3/3] Procesando: bases1.pdf

⚡ PROCESAMIENTO RÁPIDO: bases1.pdf
📄 PDF: bases1.pdf (100 páginas)
   📦 Chunk 1: páginas 1-10
   📦 Chunk 2: páginas 11-20
   📦 Chunk 3: páginas 21-30
   📦 Chunk 4: páginas 31-40
   📦 Chunk 5: páginas 41-50
   📦 Chunk 6: páginas 51-60
   📦 Chunk 7: páginas 61-70
   📦 Chunk 8: páginas 71-80
   📦 Chunk 9: páginas 81-90
   📦 Chunk 10: páginas 91-100

⚡ Procesando 10 chunks con 8 workers


Procesando chunks:   0%|          | 0/10 [00:00<?, ?it/s]


📊 Resultados:
   ✅ Exitosos: 9/10
   💾 Desde caché: 0
   ❌ Fallidos: 1
   ⏱️ Tiempo: 138.5s (13.8s/chunk)
💾 Guardado: bases1_texto.txt

✅ Completado en 138.8s
   ✅ 200,890 caracteres
   ⏱️ 138.8s

📊 RESUMEN FINAL
✅ Exitosos: 3/3
⏱️ Tiempo total: 516.2s
⚡ Promedio: 172.1s por archivo


,Archivo,Tamaño MB,Caracteres,Códigos MOP,Tiempo (s),Chunks Cache
0,bases2.pdf,7.6,338247,0,171.2,0
1,bases3.pdf,17.3,308486,100,206.2,0
2,bases1.pdf,12.2,200890,27,138.8,0



💾 Resumen guardado: storage/projects/conservacion_caminos/results/resumen_optimizado_20250905_1219.xlsx


In [ ]:
# ============================================================================
# BUDGET ANALYZER - ANÁLISIS PRESUPUESTARIO MOP CON CLAUDE (OPTIMIZADO)
# ============================================================================

import anthropic
import json
import re
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional
import pandas as pd
from IPython.display import display, HTML, Markdown
import time
import asyncio

# ============================================================================
# CONFIGURACIÓN
# ============================================================================

# Cliente Anthropic
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

# Modelo a usar (Claude Sonnet 4)
MODEL = "claude-3-5-haiku-20241022"

# Rate limiting
TOKENS_PER_MINUTE_LIMIT = 25000  # Límite conservador (5k menos que el máximo)
DELAY_BETWEEN_REQUESTS = 60  # 60 segundos entre requests pesados

# ============================================================================
# ANALIZADOR DE PRESUPUESTOS MOP OPTIMIZADO
# ============================================================================

class MOPBudgetAnalyzerOptimized:
    """
    Analizador optimizado con rate limiting y manejo de tokens.
    """
    
    def __init__(self, client: anthropic.Anthropic, model: str = MODEL):
        self.client = client
        self.model = model
        self.expected_total = 718998624  # Total esperado para validación
        self.last_request_time = 0
        self.tokens_used_this_minute = 0
        self.minute_start = time.time()
        
    def _check_rate_limit(self, estimated_tokens: int):
        """
        Verifica y espera si es necesario para respetar rate limits.
        """
        current_time = time.time()
        
        # Reset contador cada minuto
        if current_time - self.minute_start > 60:
            self.tokens_used_this_minute = 0
            self.minute_start = current_time
        
        # Si excedería el límite, esperar
        if self.tokens_used_this_minute + estimated_tokens > TOKENS_PER_MINUTE_LIMIT:
            wait_time = 60 - (current_time - self.minute_start) + 5  # +5 segundos de buffer
            print(f"⏳ Rate limit alcanzado. Esperando {wait_time:.1f}s...")
            time.sleep(wait_time)
            self.tokens_used_this_minute = 0
            self.minute_start = time.time()
        
        # Delay adicional entre requests
        time_since_last = current_time - self.last_request_time
        if time_since_last < DELAY_BETWEEN_REQUESTS:
            sleep_time = DELAY_BETWEEN_REQUESTS - time_since_last
            print(f"⏳ Esperando delay entre requests: {sleep_time:.1f}s")
            time.sleep(sleep_time)
        
        self.last_request_time = time.time()
        self.tokens_used_this_minute += estimated_tokens

    def create_optimized_prompt(self, text: str, filename: str) -> str:
        """
        Crea un prompt inteligente que identifica el tipo de documento.
        """
        # Truncar texto inteligentemente
        max_chars = 50000
        
        if len(text) > max_chars:
            # Buscar secciones importantes
            lines = text.split('\n')
            important_lines = []
            char_count = 0
            
            # Keywords ampliados para diferentes tipos de documentos MOP
            keywords = [
                'presupuesto', 'item', 'código', 'mop', 'total', 'precio', 'cantidad', 'designación',
                'proyecto', 'conservación', 'caminos', 'comunas', 'región', 'provincia',
                'especificaciones', 'bases', 'obras públicas', 'contrato', 'licitación'
            ]
            
            for line in lines[:2000]:  # Revisar más líneas
                if char_count > max_chars:
                    break
                    
                line_lower = line.lower()
                if any(keyword in line_lower for keyword in keywords) or len(line) > 80:
                    important_lines.append(line)
                    char_count += len(line) + 1
                elif len(important_lines) < 50:  # Más contexto
                    important_lines.append(line)
                    char_count += len(line) + 1
            
            text = '\n'.join(important_lines)
        
        return f"""Analiza este documento MOP chileno e identifica toda la información del proyecto.

ARCHIVO: {filename}

DOCUMENTO:
{text}

INSTRUCCIONES:
1. IDENTIFICA el tipo de documento (presupuesto, bases administrativas, especificaciones, etc.)
2. EXTRAE información del proyecto: nombre completo, ubicación, tipo de obra
3. Si hay presupuesto: busca tablas con códigos MOP (7.XXX.XXX) y totales
4. Si es bases/especificaciones: extrae información técnica del proyecto
5. REGIÓN esperada: Los Ríos

RESPONDE SOLO JSON:
{{
  "tipo_documento": "presupuesto|bases_administrativas|especificaciones|otro",
  "proyecto": {{
    "nombre": "nombre completo extraído del documento",
    "region": "región identificada",
    "provincia": "provincia si se menciona",
    "comunas": ["lista de comunas mencionadas"],
    "tipo_obra": "tipo específico de obra",
    "etapa": "etapa del proyecto si se menciona",
    "mandante": "entidad responsable"
  }},
  "presupuesto": {{
    "tiene_datos_presupuestarios": true,
    "total_neto": 0,
    "iva": 0,
    "total_con_iva": 0,
    "moneda": "CLP"
  }},
  "items_presupuestarios": [
    {{
      "codigo_mop": "código si existe",
      "descripcion": "descripción del item",
      "cantidad": 0,
      "unidad": "unidad",
      "precio_unitario": 0,
      "total": 0
    }}
  ],
  "especificaciones_tecnicas": {{
    "participacion_ciudadana": true,
    "gestion_calidad": true,
    "otras_especificaciones": ["lista de especificaciones encontradas"]
  }},
  "metadata": {{
    "items_extraidos": 0,
    "confianza_extraccion": 0.95,
    "observaciones": ["observaciones importantes"]
  }}
}}"""
    
    def analyze_document_optimized(self, text_file: Path) -> Dict:
        """
        Analiza un documento con rate limiting optimizado.
        """
        print(f"\n🤖 Analizando con Claude Sonnet 4 (optimizado): {text_file.name}")
        print("="*70)
        
        start_time = time.time()
        
        # Leer texto
        with open(text_file, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
        
        # Estadísticas del texto
        chars = len(text)
        tokens_estimate = min(chars / 4, 20000)  # Limitamos estimación
        
        print(f"📄 Caracteres: {chars:,}")
        print(f"🎯 Tokens estimados (limitados): {tokens_estimate:,.0f}")
        
        # Verificar rate limit antes de proceder
        self._check_rate_limit(int(tokens_estimate))
        
        # Crear prompt optimizado
        prompt = self.create_optimized_prompt(text, text_file.name)
        
        try:
            # Llamar a Claude con configuración optimizada
            print("⏳ Procesando con Claude...")
            response = self.client.messages.create(
                model=self.model,
                max_tokens=4000,  # Reducido para controlar costos
                temperature=0,     # Máxima precisión
                messages=[{"role": "user", "content": prompt}]
            )
            
            # Extraer respuesta
            response_text = response.content[0].text
            
            # Parsear JSON con mejor error handling
            json_start = response_text.find('{')
            json_end = response_text.rfind('}') + 1
            
            if json_start >= 0 and json_end > json_start:
                json_text = response_text[json_start:json_end]
                
                # Limpiar JSON común problemas
                json_text = re.sub(r',\s*}', '}', json_text)  # Comas finales
                json_text = re.sub(r',\s*]', ']', json_text)  # Comas en arrays
                
                try:
                    analysis = json.loads(json_text)
                except json.JSONDecodeError as e:
                    print(f"⚠️ Error JSON: {e}")
                    print(f"JSON problemático: {json_text[:500]}...")
                    analysis = self._create_fallback_analysis(text, text_file.name)
                
                # Validar y enriquecer
                analysis = self.validate_and_enrich(analysis)
                
                # Calcular costos con tokens reales del response
                input_tokens = len(prompt) / 4
                output_tokens = len(response_text) / 4
                input_cost = (input_tokens / 1_000_000) * 3.0
                output_cost = (output_tokens / 1_000_000) * 15.0
                total_cost = input_cost + output_cost
                
                elapsed = time.time() - start_time
                
                print(f"✅ Análisis completado")
                print(f"   ⏱️ Tiempo: {elapsed:.1f}s")
                print(f"   💰 Costo: ${total_cost:.4f}")
                print(f"   📊 Items extraídos: {len(analysis.get('items', []))}")
                
                # Guardar resultado
                output_file = RESULTS_DIR / f"{text_file.stem}_analisis_optimized.json"
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(analysis, f, indent=2, ensure_ascii=False)
                
                print(f"   💾 Guardado: {output_file.name}")
                
                return {
                    "success": True,
                    "analysis": analysis,
                    "file": text_file.name,
                    "cost": total_cost,
                    "time": elapsed,
                    "tokens": {"input": input_tokens, "output": output_tokens}
                }
                
            else:
                raise ValueError("No se pudo extraer JSON de la respuesta")
                
        except Exception as e:
            print(f"❌ Error: {e}")
            
            # Si es rate limit, dar información específica
            if "rate_limit" in str(e).lower():
                print("🔴 Rate limit alcanzado. Recomendaciones:")
                print("   - Esperar 1 minuto antes del próximo análisis")
                print("   - Reducir tamaño de documentos")
                print("   - Usar analyze_batch_with_delays() para múltiples archivos")
            
            return {
                "success": False,
                "error": str(e),
                "file": text_file.name,
                "error_type": "rate_limit" if "rate_limit" in str(e).lower() else "processing"
            }
    
    def _create_fallback_analysis(self, text: str, filename: str) -> Dict:
        """
        Crea análisis básico cuando falla el parsing JSON usando patrones de texto.
        """
        # Buscar información del proyecto en el texto
        project_patterns = {
            'conservacion': r'conservaci[oó]n.*?de.*?caminos',
            'comunas': r'comunas?\s+de\s+([^,\n]+)',
            'region': r'regi[oó]n\s+de\s+([^,\n]+)',
            'provincia': r'provincia\s+del?\s+([^,\n]+)'
        }
        
        # Extraer información usando patrones
        text_lower = text.lower()
        
        # Buscar nombre del proyecto
        proyecto_nombre = "Documento MOP"
        if 'conservación' in text_lower and 'caminos' in text_lower:
            if 'comunidades indígenas' in text_lower:
                proyecto_nombre = "Conservación de caminos de acceso a comunidades indígenas"
        
        # Buscar comunas
        comunas = []
        for match in re.finditer(r'comunas?\s+de\s+([^,\n.]+)', text_lower):
            comuna = match.group(1).strip().title()
            if comuna not in comunas:
                comunas.append(comuna)
        
        # Si no encuentra comunas específicas, buscar nombres conocidos
        if not comunas:
            comunas_conocidas = ['Lago Ranco', 'Futrono', 'Valdivia', 'La Unión', 'Río Bueno']
            for comuna in comunas_conocidas:
                if comuna.lower() in text_lower:
                    comunas.append(comuna)
        
        # Determinar tipo de documento
        tipo_doc = "documento_mop"
        if 'presupuesto' in text_lower and 'precio' in text_lower:
            tipo_doc = "presupuesto"
        elif 'bases administrativas' in text_lower:
            tipo_doc = "bases_administrativas"
        elif 'especificaciones' in text_lower:
            tipo_doc = "especificaciones"
        
        # Buscar códigos MOP o items presupuestarios
        items_encontrados = []
        codigo_pattern = r'7\.\d{3}\.\d{3}'
        codigos = re.findall(codigo_pattern, text)
        
        for codigo in codigos[:10]:  # Limitar a 10 items
            items_encontrados.append({
                "codigo_mop": codigo,
                "descripcion": "Item extraído del documento",
                "cantidad": 0,
                "unidad": "ST",
                "precio_unitario": 0,
                "total": 0
            })
        
        return {
            "tipo_documento": tipo_doc,
            "proyecto": {
                "nombre": proyecto_nombre,
                "region": "Los Ríos",
                "provincia": "Del Ranco" if 'ranco' in text_lower else "Los Ríos",
                "comunas": comunas if comunas else ["Por determinar"],
                "tipo_obra": "Conservación de caminos",
                "etapa": "Análisis de documento",
                "mandante": "MOP - Ministerio de Obras Públicas"
            },
            "presupuesto": {
                "tiene_datos_presupuestarios": len(items_encontrados) > 0,
                "total_neto": 0,
                "iva": 0,
                "total_con_iva": 0,
                "moneda": "CLP"
            },
            "items_presupuestarios": items_encontrados,
            "especificaciones_tecnicas": {
                "participacion_ciudadana": "participación ciudadana" in text_lower,
                "gestion_calidad": "gestión de la calidad" in text_lower or "calidad" in text_lower,
                "otras_especificaciones": self._extract_specifications(text_lower)
            },
            "metadata": {
                "items_extraidos": len(items_encontrados),
                "confianza_extraccion": 0.6,
                "es_fallback": True,
                "observaciones": [f"Análisis fallback aplicado a {filename}"]
            }
        }
    
    def _extract_specifications(self, text_lower: str) -> List[str]:
        """Extrae especificaciones técnicas del texto."""
        specs = []
        
        spec_keywords = [
            "participación ciudadana",
            "gestión de la calidad", 
            "especificaciones ambientales",
            "consulta indígena",
            "bases administrativas",
            "términos de referencia"
        ]
        
        for spec in spec_keywords:
            if spec in text_lower:
                specs.append(spec.title())
        
        return specs

    def validate_and_enrich(self, analysis: Dict) -> Dict:
        """
        Valida y enriquece análisis con la nueva estructura.
        """
        # Convertir estructura nueva a formato compatible con reportes existentes
        
        # Validar estructura básica
        if not analysis.get("proyecto"):
            analysis["proyecto"] = {"nombre": "Proyecto no identificado"}
        
        # Convertir items_presupuestarios a items (para compatibilidad)
        items_presupuestarios = analysis.get("items_presupuestarios", [])
        analysis["items"] = items_presupuestarios
        
        # Calcular total de items
        total_calculado = sum(item.get('total', 0) for item in items_presupuestarios)
        
        # Enriquecer presupuesto
        presupuesto = analysis.get("presupuesto", {})
        if total_calculado > 0:
            presupuesto['total_calculado'] = total_calculado
            presupuesto['total_con_iva'] = total_calculado
        
        # Validación específica
        tiene_datos = presupuesto.get("tiene_datos_presupuestarios", False)
        if tiene_datos and total_calculado > 0:
            diferencia = abs(total_calculado - self.expected_total)
            pct_diferencia = (diferencia / self.expected_total) * 100 if self.expected_total > 0 else 100
            
            presupuesto['validacion'] = {
                'total_esperado': self.expected_total,
                'diferencia': diferencia,
                'porcentaje_diferencia': round(pct_diferencia, 2),
                'es_valido': pct_diferencia < 10
            }
        else:
            # Para documentos sin presupuesto (bases, especificaciones)
            presupuesto['validacion'] = {
                'total_esperado': 0,
                'diferencia': 0,
                'porcentaje_diferencia': 0,
                'es_valido': True  # Válido porque no es un documento presupuestario
            }
        
        analysis["presupuesto"] = presupuesto
        
        # Agregar metadata
        metadata = analysis.get('metadata', {})
        metadata['timestamp_analisis'] = datetime.now().isoformat()
        metadata['modelo_usado'] = self.model
        metadata['tipo_documento'] = analysis.get('tipo_documento', 'desconocido')
        
        analysis['metadata'] = metadata
        
        return analysis

    def analyze_batch_with_delays(self, text_files: List[Path]) -> List[Dict]:
        """
        Analiza múltiples archivos con delays automáticos entre cada uno.
        """
        results = []
        total_cost = 0
        total_time = 0
        
        print(f"\n🚀 ANÁLISIS BATCH DE {len(text_files)} ARCHIVOS")
        print("="*60)
        print(f"⏰ Tiempo estimado: {len(text_files) * 2:.1f} minutos")
        print("="*60)
        
        for i, text_file in enumerate(text_files, 1):
            print(f"\n📋 Archivo {i}/{len(text_files)}")
            
            result = self.analyze_document_optimized(text_file)
            results.append(result)
            
            if result['success']:
                total_cost += result['cost']
                total_time += result['time']
                
                # Generar reporte HTML simplificado
                html_report = self.generate_simple_html_report(result['analysis'])
                
                # Guardar HTML
                html_file = RESULTS_DIR / f"{text_file.stem}_reporte_optimized.html"
                with open(html_file, 'w', encoding='utf-8') as f:
                    f.write(html_report)
                
                print(f"   📄 Reporte HTML: {html_file.name}")
            
            # Delay entre archivos (excepto el último)
            if i < len(text_files):
                print(f"⏳ Esperando {DELAY_BETWEEN_REQUESTS}s antes del siguiente archivo...")
                time.sleep(DELAY_BETWEEN_REQUESTS)
        
        # Resumen final
        print(f"\n" + "="*60)
        print(f"📊 RESUMEN BATCH")
        print(f"="*60)
        successful = len([r for r in results if r['success']])
        print(f"✅ Exitosos: {successful}/{len(text_files)}")
        print(f"⏱️ Tiempo total: {total_time/60:.1f} min")
        print(f"💰 Costo total: ${total_cost:.4f} USD")
        
        return results

    def generate_simple_html_report(self, analysis: Dict) -> str:
        """
        Genera reporte HTML adaptado al tipo de documento.
        """
        proyecto = analysis.get('proyecto', {})
        presupuesto = analysis.get('presupuesto', {})
        items = analysis.get('items', [])
        tipo_doc = analysis.get('tipo_documento', 'documento_mop')
        specs = analysis.get('especificaciones_tecnicas', {})
        
        # Título según tipo de documento
        titulo_doc = {
            'presupuesto': 'Análisis Presupuestario',
            'bases_administrativas': 'Análisis de Bases Administrativas', 
            'especificaciones': 'Análisis de Especificaciones Técnicas',
            'documento_mop': 'Análisis de Documento MOP'
        }.get(tipo_doc, 'Análisis de Documento MOP')
        
        html = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>{titulo_doc} - {proyecto.get('nombre', 'Proyecto')}</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; line-height: 1.6; }}
        .header {{ background: #2c3e50; color: white; padding: 20px; border-radius: 5px; margin-bottom: 20px; }}
        .section {{ margin: 20px 0; padding: 15px; border: 1px solid #ddd; border-radius: 5px; background: #f8f9fa; }}
        .presupuesto {{ background: #e8f5e8; }}
        .especificaciones {{ background: #e8f0ff; }}
        .warning {{ background: #fff3cd; border-color: #ffeaa7; }}
        .total {{ font-size: 1.5em; color: #27ae60; font-weight: bold; }}
        .no-presupuesto {{ font-size: 1.2em; color: #e67e22; font-weight: bold; }}
        table {{ width: 100%; border-collapse: collapse; margin-top: 10px; }}
        th, td {{ padding: 8px; border: 1px solid #ddd; text-align: left; }}
        th {{ background: #f8f9fa; font-weight: bold; }}
        .items-table {{ max-height: 400px; overflow-y: auto; }}
        .badge {{ display: inline-block; padding: 3px 8px; border-radius: 3px; font-size: 0.9em; }}
        .badge-success {{ background: #d4edda; color: #155724; }}
        .badge-warning {{ background: #fff3cd; color: #856404; }}
        .badge-info {{ background: #d1ecf1; color: #0c5460; }}
    </style>
</head>
<body>
    <div class="header">
        <h1>📊 {titulo_doc}</h1>
        <p><strong>{proyecto.get('nombre', 'Proyecto MOP')}</strong></p>
        <span class="badge badge-info">Tipo: {tipo_doc.replace('_', ' ').title()}</span>
    </div>
    
    <div class="section">
        <h2>📍 Información del Proyecto</h2>
        <table>
            <tr><td><strong>Nombre Completo:</strong></td><td>{proyecto.get('nombre', 'N/D')}</td></tr>
            <tr><td><strong>Región:</strong></td><td>{proyecto.get('region', 'N/D')}</td></tr>
            <tr><td><strong>Provincia:</strong></td><td>{proyecto.get('provincia', 'N/D')}</td></tr>
            <tr><td><strong>Comunas:</strong></td><td>{', '.join(proyecto.get('comunas', ['N/D']))}</td></tr>
            <tr><td><strong>Tipo de Obra:</strong></td><td>{proyecto.get('tipo_obra', 'N/D')}</td></tr>
            <tr><td><strong>Etapa:</strong></td><td>{proyecto.get('etapa', 'N/D')}</td></tr>
            <tr><td><strong>Mandante:</strong></td><td>{proyecto.get('mandante', 'N/D')}</td></tr>
        </table>
    </div>"""
        
        # Sección de presupuesto (solo si hay datos)
        tiene_presupuesto = presupuesto.get('tiene_datos_presupuestarios', False)
        if tiene_presupuesto and len(items) > 0:
            html += f"""
    <div class="section presupuesto">
        <h2>💰 Información Presupuestaria</h2>
        <p class="total">Total Identificado: ${presupuesto.get('total_con_iva', 0):,.0f} CLP</p>
        <table>
            <tr><td><strong>Total Neto:</strong></td><td>${presupuesto.get('total_neto', 0):,.0f} CLP</td></tr>
            <tr><td><strong>IVA (19%):</strong></td><td>${presupuesto.get('iva', 0):,.0f} CLP</td></tr>
            <tr><td><strong>Total con IVA:</strong></td><td>${presupuesto.get('total_con_iva', 0):,.0f} CLP</td></tr>
        </table>
    </div>
    
    <div class="section">
        <h2>📝 Items Presupuestarios ({len(items)} items)</h2>
        <div class="items-table">
            <table>
                <thead>
                    <tr>
                        <th>Código MOP</th>
                        <th>Descripción</th>
                        <th>Cantidad</th>
                        <th>Unidad</th>
                        <th>P.Unitario</th>
                        <th>Total</th>
                    </tr>
                </thead>
                <tbody>"""
            
            for item in items[:30]:
                html += f"""
                    <tr>
                        <td>{item.get('codigo_mop', 'N/D')}</td>
                        <td>{item.get('descripcion', item.get('designacion', 'N/D'))[:60]}...</td>
                        <td>{item.get('cantidad', 0):,.2f}</td>
                        <td>{item.get('unidad', 'N/D')}</td>
                        <td>${item.get('precio_unitario', 0):,.0f}</td>
                        <td>${item.get('total', 0):,.0f}</td>
                    </tr>"""
            
            if len(items) > 30:
                html += f"""
                    <tr style="background: #fff3cd;">
                        <td colspan="6" style="text-align: center;">
                            ⚠️ Mostrando 30 de {len(items)} items totales
                        </td>
                    </tr>"""
            
            html += """
                </tbody>
            </table>
        </div>
    </div>"""
        else:
            html += f"""
    <div class="section warning">
        <h2>💰 Información Presupuestaria</h2>
        <p class="no-presupuesto">Este documento no contiene datos presupuestarios detallados</p>
        <p>Tipo de documento: <strong>{tipo_doc.replace('_', ' ').title()}</strong></p>
        <p>Para análisis presupuestario, se requiere el documento de presupuesto oficial del proyecto.</p>
    </div>"""
        
        # Sección de especificaciones técnicas
        if specs:
            html += f"""
    <div class="section especificaciones">
        <h2>📋 Especificaciones Técnicas Identificadas</h2>
        <table>
            <tr>
                <td><strong>Participación Ciudadana:</strong></td>
                <td>{'✅ Sí' if specs.get('participacion_ciudadana') else '❌ No'}</td>
            </tr>
            <tr>
                <td><strong>Gestión de Calidad:</strong></td>
                <td>{'✅ Sí' if specs.get('gestion_calidad') else '❌ No'}</td>
            </tr>
        </table>"""
            
            otras_specs = specs.get('otras_especificaciones', [])
            if otras_specs:
                html += """
        <h3>Otras Especificaciones:</h3>
        <ul>"""
                for spec in otras_specs:
                    html += f"<li>{spec}</li>"
                html += "</ul>"
            
            html += "</div>"
        
        # Información del análisis
        metadata = analysis.get('metadata', {})
        confianza = metadata.get('confianza_extraccion', metadata.get('confianza', 0))
        
        html += f"""
    <div class="section">
        <h3>ℹ️ Información del Análisis</h3>
        <table>
            <tr><td><strong>Fecha:</strong></td><td>{datetime.now().strftime('%d/%m/%Y %H:%M')}</td></tr>
            <tr><td><strong>Tipo de Documento:</strong></td><td>{tipo_doc.replace('_', ' ').title()}</td></tr>
            <tr><td><strong>Items Extraídos:</strong></td><td>{metadata.get('items_extraidos', len(items))}</td></tr>
            <tr><td><strong>Confianza:</strong></td><td>{confianza*100:.1f}%</td></tr>
            <tr><td><strong>Método:</strong></td><td>{'Análisis Fallback' if metadata.get('es_fallback') else 'Análisis Claude'}</td></tr>
        </table>"""
        
        observaciones = metadata.get('observaciones', [])
        if observaciones:
            html += "<h4>Observaciones:</h4><ul>"
            for obs in observaciones:
                html += f"<li>{obs}</li>"
            html += "</ul>"
        
        html += """
    </div>
</body>
</html>"""
        
        return html

    def quick_document_analysis(self, text: str, filename: str) -> Dict:
        """
        Análisis rápido para identificar tipo de documento y contenido básico.
        """
        text_lower = text.lower()
        
        # Detectar tipo de documento
        doc_type = "documento_mop"
        if "presupuesto oficial" in text_lower or ("precio" in text_lower and "total" in text_lower and "item" in text_lower):
            doc_type = "presupuesto"
        elif "bases administrativas" in text_lower:
            doc_type = "bases_administrativas"  
        elif "especificaciones" in text_lower and ("técnicas" in text_lower or "ambientales" in text_lower):
            doc_type = "especificaciones"
        
        # Extraer información básica del proyecto
        proyecto_info = self._extract_project_info(text)
        
        # Buscar códigos MOP
        codigos_mop = re.findall(r'7\.\d{3}\.\d{3}', text)
        
        # Buscar totales monetarios
        totales = re.findall(r'\$?\s*(\d{1,3}(?:\.\d{3})+)', text)
        totales_numericos = [int(t.replace('.', '')) for t in totales if len(t.replace('.', '')) >= 6]
        
        return {
            "tipo_documento": doc_type,
            "proyecto_detectado": proyecto_info,
            "codigos_mop_encontrados": len(codigos_mop),
            "totales_monetarios": totales_numericos[:5],  # Primeros 5 totales
            "tiene_datos_presupuestarios": len(codigos_mop) > 0 or (doc_type == "presupuesto"),
            "confianza_deteccion": self._calculate_confidence(doc_type, len(codigos_mop), proyecto_info)
        }
    
    def _extract_project_info(self, text: str) -> Dict:
        """Extrae información básica del proyecto del texto."""
        text_lower = text.lower()
        
        info = {
            "nombre": "",
            "region": "",
            "comunas": [],
            "tipo_obra": ""
        }
        
        # Buscar nombre del proyecto
        if "conservación" in text_lower and "caminos" in text_lower:
            if "comunidades indígenas" in text_lower:
                info["nombre"] = "Conservación de caminos de acceso a comunidades indígenas"
            else:
                info["nombre"] = "Conservación de caminos"
        
        # Buscar región
        region_match = re.search(r'región\s+de\s+([^,\n.]+)', text_lower)
        if region_match:
            info["region"] = region_match.group(1).strip().title()
        elif "los ríos" in text_lower:
            info["region"] = "Los Ríos"
        
        # Buscar comunas
        comunas_patterns = [
            r'comunas?\s+de\s+([^,\n.]+)',
            r'lago\s+ranco',
            r'futrono',
            r'valdivia'
        ]
        
        for pattern in comunas_patterns:
            matches = re.findall(pattern, text_lower)
            for match in matches:
                comuna = match.strip().title()
                if comuna and comuna not in info["comunas"]:
                    info["comunas"].append(comuna)
        
        # Buscar tipo de obra
        if "conservación" in text_lower:
            info["tipo_obra"] = "Conservación"
        elif "construcción" in text_lower:
            info["tipo_obra"] = "Construcción"
        elif "mejoramiento" in text_lower:
            info["tipo_obra"] = "Mejoramiento"
        
        return info
    
    def _calculate_confidence(self, doc_type: str, codigos_count: int, proyecto_info: Dict) -> float:
        """Calcula la confianza de la detección."""
        confidence = 0.5  # Base
        
        # Bonus por tipo de documento claro
        if doc_type != "documento_mop":
            confidence += 0.2
        
        # Bonus por códigos MOP encontrados
        if codigos_count > 0:
            confidence += min(0.3, codigos_count * 0.05)
        
        # Bonus por información del proyecto
        if proyecto_info.get("nombre"):
            confidence += 0.2
        if proyecto_info.get("region"):
            confidence += 0.1
        if proyecto_info.get("comunas"):
            confidence += 0.1
        
        return min(1.0, confidence)

# ============================================================================
# FUNCIONES PRINCIPALES OPTIMIZADAS
# ============================================================================

def analyze_mop_budgets_optimized():
    """
    Analiza todos los documentos MOP con rate limiting optimizado.
    """
    print("\n" + "="*80)
    print("🚀 ANÁLISIS OPTIMIZADO DE PRESUPUESTOS MOP")
    print("="*80)
    
    # Buscar archivos de texto
    text_files = list(RESULTS_DIR.glob("*_texto.txt"))
    
    if not text_files:
        print("❌ No hay archivos de texto para analizar")
        return None
    
    print(f"\n📚 Archivos encontrados: {len(text_files)}")
    for f in text_files:
        size_kb = f.stat().st_size / 1024
        print(f"   - {f.name} ({size_kb:.1f} KB)")
    
    # Confirmar procesamiento
    response = input(f"\n¿Proceder con análisis de {len(text_files)} archivos? (s/n): ")
    if response.lower() != 's':
        print("❌ Análisis cancelado")
        return None
    
    # Inicializar analizador optimizado
    analyzer = MOPBudgetAnalyzerOptimized(client)
    
    # Análisis con delays automáticos
    results = analyzer.analyze_batch_with_delays(text_files)
    
    # Crear tabla resumen mejorada
    summary_data = []
    for r in results:
        if r['success']:
            analysis = r['analysis']
            proyecto = analysis.get('proyecto', {})
            presupuesto = analysis.get('presupuesto', {})
            metadata = analysis.get('metadata', {})
            
            # Determinar si tiene datos presupuestarios
            tiene_presupuesto = presupuesto.get('tiene_datos_presupuestarios', False)
            tipo_doc = analysis.get('tipo_documento', 'documento_mop')
            
            summary_data.append({
                'Archivo': r['file'],
                'Tipo Documento': tipo_doc.replace('_', ' ').title(),
                'Proyecto': proyecto.get('nombre', 'N/D')[:60],
                'Región': proyecto.get('region', 'N/D'),
                'Comunas': ', '.join(proyecto.get('comunas', ['N/D'])[:2]),
                'Items': len(analysis.get('items', [])),
                'Total Presupuesto': f"${presupuesto.get('total_con_iva', 0):,.0f}" if tiene_presupuesto else 'Sin datos',
                'Tiene Presupuesto': 'Sí' if tiene_presupuesto else 'No',
                'Confianza': f"{metadata.get('confianza_extraccion', metadata.get('confianza', 0))*100:.1f}%",
                'Método': 'Fallback' if metadata.get('es_fallback') else 'Claude',
                'Costo': f"${r['cost']:.4f}",
                'Tiempo': f"{r['time']:.1f}s"
            })
        else:
            error_type = r.get('error_type', 'unknown')
            summary_data.append({
                'Archivo': r['file'],
                'Tipo Documento': 'ERROR',
                'Proyecto': f'Error: {error_type}',
                'Región': 'N/A',
                'Comunas': 'N/A',
                'Items': 0,
                'Total Presupuesto': '$0',
                'Tiene Presupuesto': 'No',
                'Confianza': '0%',
                'Método': 'Error',
                'Costo': '$0',
                'Tiempo': '0s'
            })
    
    if summary_data:
        df_summary = pd.DataFrame(summary_data)
        display(df_summary)
        
        # Guardar Excel con resumen
        excel_file = RESULTS_DIR / f"analisis_optimized_resumen_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
        df_summary.to_excel(excel_file, index=False)
        print(f"\n💾 Resumen guardado en: {excel_file}")
    
    return results

def analyze_single_optimized(filename: str):
    """
    Analiza un único documento con optimizaciones.
    """
    text_file = RESULTS_DIR / filename
    
    if not text_file.exists():
        print(f"❌ Archivo no encontrado: {filename}")
        return None
    
    analyzer = MOPBudgetAnalyzerOptimized(client)
    result = analyzer.analyze_document_optimized(text_file)
    
    if result['success']:
        # Generar y mostrar reporte HTML
        html_report = analyzer.generate_simple_html_report(result['analysis'])
        display(HTML(html_report))
        
        # Guardar HTML
        html_file = RESULTS_DIR / f"{text_file.stem}_reporte_optimized.html"
        with open(html_file, 'w', encoding='utf-8') as f:
            f.write(html_report)
        
        print(f"\n✅ Reporte HTML guardado: {html_file}")
    
    return result

def analyze_mop_budgets_smart():
    """
    Análisis inteligente que primero identifica el tipo de documento.
    """
    print("\n" + "="*80)
    print("🚀 ANÁLISIS INTELIGENTE DE DOCUMENTOS MOP")
    print("="*80)
    
    # Buscar archivos de texto
    text_files = list(RESULTS_DIR.glob("*_texto.txt"))
    
    if not text_files:
        print("❌ No hay archivos de texto para analizar")
        return None
    
    print(f"\n📚 Archivos encontrados: {len(text_files)}")
    
    # Inicializar analizador
    analyzer = MOPBudgetAnalyzerOptimized(client)
    
    # Análisis rápido primero
    print("\n🔍 FASE 1: Análisis rápido de documentos")
    print("-" * 50)
    
    quick_results = []
    for text_file in text_files:
        with open(text_file, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
        
        quick_analysis = analyzer.quick_document_analysis(text, text_file.name)
        quick_results.append({
            'file': text_file,
            'analysis': quick_analysis
        })
        
        tipo = quick_analysis['tipo_documento']
        confianza = quick_analysis['confianza_deteccion']
        codigos = quick_analysis['codigos_mop_encontrados']
        
        print(f"   📄 {text_file.name}")
        print(f"      Tipo: {tipo.replace('_', ' ').title()}")
        print(f"      Códigos MOP: {codigos}")
        print(f"      Confianza: {confianza*100:.1f}%")
        print(f"      Proyecto: {quick_analysis['proyecto_detectado'].get('nombre', 'No identificado')[:50]}")
    
    # Mostrar resumen de análisis rápido
    print(f"\n📊 RESUMEN ANÁLISIS RÁPIDO:")
    tipos_doc = {}
    for qr in quick_results:
        tipo = qr['analysis']['tipo_documento']
        tipos_doc[tipo] = tipos_doc.get(tipo, 0) + 1
    
    for tipo, count in tipos_doc.items():
        print(f"   - {tipo.replace('_', ' ').title()}: {count} archivo(s)")
    
    # Preguntar si continuar con análisis completo
    response = input(f"\n¿Continuar con análisis completo de {len(text_files)} archivos? (s/n): ")
    if response.lower() != 's':
        print("❌ Análisis cancelado")
        return quick_results
    
    # FASE 2: Análisis completo con Claude
    print(f"\n🤖 FASE 2: Análisis completo con Claude")
    print("-" * 50)
    
    results = analyzer.analyze_batch_with_delays([qr['file'] for qr in quick_results])
    
    # Combinar resultados
    combined_results = []
    for i, result in enumerate(results):
        combined_result = result.copy()
        combined_result['quick_analysis'] = quick_results[i]['analysis']
        combined_results.append(combined_result)
    
    return combined_results

def analyze_single_smart(filename: str):
    """
    Análisis inteligente de un solo documento.
    """
    text_file = RESULTS_DIR / filename
    
    if not text_file.exists():
        print(f"❌ Archivo no encontrado: {filename}")
        return None
    
    analyzer = MOPBudgetAnalyzerOptimized(client)
    
    # Análisis rápido primero
    print("🔍 Análisis rápido...")
    with open(text_file, 'r', encoding='utf-8', errors='ignore') as f:
        text = f.read()
    
    quick_analysis = analyzer.quick_document_analysis(text, filename)
    
    print(f"📄 Tipo de documento: {quick_analysis['tipo_documento'].replace('_', ' ').title()}")
    print(f"🎯 Códigos MOP encontrados: {quick_analysis['codigos_mop_encontrados']}")
    print(f"📊 Confianza: {quick_analysis['confianza_deteccion']*100:.1f}%")
    
    proyecto = quick_analysis['proyecto_detectado']
    if proyecto.get('nombre'):
        print(f"🏗️ Proyecto: {proyecto['nombre']}")
        print(f"📍 Región: {proyecto.get('region', 'No especificada')}")
        if proyecto.get('comunas'):
            print(f"🏘️ Comunas: {', '.join(proyecto['comunas'])}")
    
    # Análisis completo
    print(f"\n🤖 Análisis completo con Claude...")
    result = analyzer.analyze_document_optimized(text_file)
    
    if result['success']:
        # Mostrar reporte
        html_report = analyzer.generate_simple_html_report(result['analysis'])
        display(HTML(html_report))
        
        # Guardar HTML
        html_file = RESULTS_DIR / f"{text_file.stem}_reporte_smart.html"
        with open(html_file, 'w', encoding='utf-8') as f:
            f.write(html_report)
        
        print(f"\n✅ Reporte HTML guardado: {html_file}")
    
    # Combinar resultados
    result['quick_analysis'] = quick_analysis
    return result

print("\n" + "="*80)
print("✅ BUDGET ANALYZER MOP INTELIGENTE CARGADO")
print("="*80)
print("\nFunciones principales:")
print("  • analyze_mop_budgets_smart() - Análisis inteligente con detección de tipo")
print("  • analyze_single_smart('bases1_texto.txt') - Análisis individual inteligente")
print("\nFunciones optimizadas:")
print("  • analyze_mop_budgets_optimized() - Análisis con rate limiting")
print("  • analyze_single_optimized('archivo.txt') - Análisis individual optimizado")
print("\n🧠 CARACTERÍSTICAS INTELIGENTES:")
print("   - Detección automática del tipo de documento")
print("   - Análisis rápido previo antes del análisis completo")
print("   - Extracción mejorada de información del proyecto")
print("   - Manejo específico según tipo de documento")
print("   - Rate limiting automático")
print("\n💡 Recomendado: results = analyze_mop_budgets_smart()")
print("="*80)


✅ BUDGET ANALYZER MOP INTELIGENTE CARGADO

Funciones principales:
  • analyze_mop_budgets_smart() - Análisis inteligente con detección de tipo
  • analyze_single_smart('bases1_texto.txt') - Análisis individual inteligente

Funciones optimizadas:
  • analyze_mop_budgets_optimized() - Análisis con rate limiting
  • analyze_single_optimized('archivo.txt') - Análisis individual optimizado

🧠 CARACTERÍSTICAS INTELIGENTES:
   - Detección automática del tipo de documento
   - Análisis rápido previo antes del análisis completo
   - Extracción mejorada de información del proyecto
   - Manejo específico según tipo de documento
   - Rate limiting automático

💡 Recomendado: results = analyze_mop_budgets_smart()


In [14]:
results = analyze_mop_budgets()



🚀 ANÁLISIS DE PRESUPUESTOS MOP CON CLAUDE SONNET 4

📚 Archivos a analizar: 3
   - bases2_texto.txt (337.1 KB)
   - bases3_texto.txt (306.5 KB)
   - bases1_texto.txt (200.4 KB)

🤖 Analizando con Claude Sonnet 4: bases2_texto.txt
📄 Caracteres: 338,247
🎯 Tokens estimados: 84,562
⏳ Procesando con Claude...
✅ Análisis completado
   ⏱️ Tiempo: 14.1s
   💰 Costo: $0.0866
   📊 Items extraídos: 0
   💾 Guardado: bases2_texto_analisis_budget.json



🤖 Analizando con Claude Sonnet 4: bases3_texto.txt
📄 Caracteres: 308,486
🎯 Tokens estimados: 77,122
⏳ Procesando con Claude...
✅ Análisis completado
   ⏱️ Tiempo: 14.7s
   💰 Costo: $0.0868
   📊 Items extraídos: 0
   💾 Guardado: bases3_texto_analisis_budget.json



🤖 Analizando con Claude Sonnet 4: bases1_texto.txt
📄 Caracteres: 200,890
🎯 Tokens estimados: 50,222
⏳ Procesando con Claude...
❌ Error: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'This request would exceed the rate limit for your organization (11cfa296-d5b1-45da-a861-e519afa2f730) of 30,000 input tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}, 'request_id': 'req_011CSqe8GMRi89v2Z6r9giKp'}

📊 RESUMEN DE ANÁLISIS
✅ Archivos procesados: 3
⏱️ Tiempo total: 28.8s
💰 Costo total: $0.1734 USD


,Archivo,Proyecto,Items,Total Presupuesto,Validación,Costo Análisis,Tiempo (s)
0,bases2_texto.txt,No se identifica proyecto específico en el doc...,0,$0,❌,$0.0866,14.1
1,bases3_texto.txt,No se identifica proyecto específico en el doc...,0,$0,❌,$0.0868,14.7



💾 Resumen guardado en: storage/projects/conservacion_caminos/results/analisis_budget_resumen_20250905_1322.xlsx


In [15]:
result = analyze_single_smart('bases1_texto.txt')


🔍 Análisis rápido...
📄 Tipo de documento: Presupuesto
🎯 Códigos MOP encontrados: 1
📊 Confianza: 100.0%
🏗️ Proyecto: Conservación de caminos de acceso a comunidades indígenas
📍 Región: Los Ríos Valdivia 2024 Yungay Nº 621
🏘️ Comunas: Lago Ranco Y Futrono, Lago Ranco Y Futrono Con Una Longitud Total De 7, Lago Ranco, Futrono, Valdivia

🤖 Análisis completo con Claude...

🤖 Analizando con Claude Sonnet 4 (optimizado): bases1_texto.txt
📄 Caracteres: 200,890
🎯 Tokens estimados (limitados): 20,000
⏳ Procesando con Claude...
✅ Análisis completado
   ⏱️ Tiempo: 44.8s
   💰 Costo: $0.0723
   📊 Items extraídos: 15
   💾 Guardado: bases1_texto_analisis_optimized.json



✅ Reporte HTML guardado: storage/projects/conservacion_caminos/results/bases1_texto_reporte_smart.html
